# Introduction

## Contexte et objectifs

Ce projet analyse un dataset Spotify contenant des caractéristiques audio de milliers de chansons provenant de différents genres musicaux. L'objectif principal est d'explorer la structure des données musicales à travers diverses techniques d'analyse multivariée et de réduction de dimension.

## Dataset

Le dataset comprend des variables quantitatives (danceability, energy, valence, tempo, etc.) et qualitatives (genre, artiste, tonalité, mode) permettant une analyse complète des profils musicaux. Après prétraitement, nous disposons de données sur les caractéristiques audio objectives et les métadonnées descriptives des morceaux.

## Approche méthodologique

Cette analyse combine plusieurs techniques complémentaires :
- **Analyse en Composantes Principales (ACP)** pour explorer les relations entre variables quantitatives et réduire la dimensionnalité
- **Analyse des Correspondances Multiples (MCA)** pour étudier les associations entre variables qualitatives (genres, tonalités, modes)
- **Analyse Factorielle Multiple (MFA)** pour intégrer simultanément données quantitatives et qualitatives dans une analyse unifiée
- **Techniques de réduction non-linéaire** (MDS, t-SNE) pour visualiser la structure complexe des données musicales
- **Factorisation Matricielle Non-négative (NMF)** pour identifier des profils musicaux latents et développer un système de recommandation
- **Algorithmes de clustering** pour segmenter les morceaux en groupes homogènes selon leurs caractéristiques audio

L'objectif est de comprendre comment les caractéristiques musicales s'organisent, identifier des patterns et profils musicaux distincts, et développer des applications pratiques comme un système de recommandation personnalisé basé sur les profils latents découverts.

# Data preprocessing

## Préparation et nettoyage des données

Le préprocessing est une étape cruciale qui conditionne la qualité des analyses statistiques ultérieures. Pour ce dataset Spotify, plusieurs transformations sont nécessaires :

### Objectifs du préprocessing

- **Élimination des variables non pertinentes** : Suppression des identifiants techniques (`track_id`, `album_id`, `playlist_id`) qui n'apportent pas d'information analytique
- **Gestion des valeurs manquantes** : Identification et traitement des données incomplètes pour éviter les biais dans les analyses
- **Standardisation des types de données** : Conversion des variables catégorielles et temporelles dans les formats appropriés
- **Déduplication intelligente** : Conservation des versions les plus populaires des morceaux dupliqués
- **Harmonisation des unités** : Conversion de la durée en secondes pour une meilleure interprétabilité

### Impact sur les analyses multivariées

Un preprocessing rigoureux garantit :
- Des résultats d'ACP non biaisés par des échelles de variables hétérogènes
- Une MCA cohérente avec des modalités catégorielles bien définies
- Des algorithmes de clustering et de réduction de dimension plus performants
- Une meilleure généralisation des modèles de recommandation

In [ ]:
# Chargement des librairies nécessaires
library(ggplot2)
library(tidyverse)
library(gridExtra)
library(GGally)
library(plotly)
library(corrplot)
library(reshape2)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)
library(ggpubr)

In [ ]:
# Lecture des données
path <- "data/"
song <- read.csv(paste0(path, "spotify_songs.csv"), header = TRUE, sep = ",")

# Premières lignes du jeu de données
head(song)

# Vérification du contenu
summary(song)

In [ ]:
# Check the data types
str(song)

In [ ]:
# Drop the track_id, track_album_id, playlist_id columns
song <- song[, -c(1, 5, 9)]

# As factor the categorical variables track_artist, playlist_genre, playlist_subgenre, key, mode, playlist_name, track_album_name
song$playlist_name <- as.factor(song$playlist_name)
song$track_album_name <- as.factor(song$track_album_name)
song$track_artist <- as.factor(song$track_artist)
song$playlist_genre <- as.factor(song$playlist_genre)
song$playlist_subgenre <- as.factor(song$playlist_subgenre)
song$key <- factor(song$key, levels = c(-1, 0:11), labels = c("No key detected", "C", "C♯/D♭", "D", "D♯/E♭", "E", "F", "F♯/G♭", "G", "G♯/A♭", "A", "A♯/B♭", "B"))
song$mode <- factor(song$mode, levels = c(0, 1), labels = c("minor", "major"))

# track_album_release_date to date (if the full date is not available, we will use the first day of the year)
song$track_album_release_date <- as.Date(ifelse(nchar(song$track_album_release_date) != 10, 
                                                paste0(substr(song$track_album_release_date, 1, 4), "-01-01"), 
                                                song$track_album_release_date), 
                                         format = "%Y-%m-%d")

# Convert the duration_ms to seconds and rename the column to duration_s
song$duration_s <- song$duration_ms / 1000
song$duration_ms <- NULL

# Check the modified dataset
summary(song)
head(song)

In [ ]:
# Check for missing values
colSums(is.na(song))

# Drop the missing values
song <- na.omit(song)

# Check the modified dataset
colSums(is.na(song))

str(song)

In [ ]:
# Check for duplicated data
cat(sum(duplicated(song)),'\n')

# Check for duplicated song names with same artist
cat(sum(duplicated(song[, c("track_name", "track_artist")])), '\n')

In [ ]:
# New data set by removing columns playlist_name, playlist_genre, playlist_subgenre
onlysongs <- song[, -c(6, 7, 8)]

# Check for duplicated data
cat(sum(duplicated(onlysongs)), '\n')

# Drop duplicates
onlysongs <- unique(onlysongs)

# Print number of songs with same name and same artist
cat(sum(duplicated(onlysongs[, c("track_name", "track_artist")])), '\n')

# print rows with name "Something real"
onlysongs[onlysongs$track_name == "Something Real", ]

# Remove duplicates with the same name and artist, keeping only one with the highest popularity
onlysongs <- onlysongs %>%
    group_by(track_name, track_artist) %>%
    filter(track_popularity == max(track_popularity)) %>%
    slice(1) %>%  # If there are ties in popularity, keep just one
    ungroup()

Dans ce pré-processing de nos données, nous avons fait différents choix pour traiter nos données qui sont les suivantes :
* Nous avons supprimé les lignes du jeu qui ne possédaient des valeurs manquantes (nom d'artiste, de morceau...), et nous avons constaté que uniquement **5 lignes** avaient des données non spécifiées. Nous avons donc fait le choix de les supprimer en supposant que cela n'aurait pas d'impact, étant donné la taille du jeu de données initial.
* Nous avons également supprimé les colonnes qui représentaient les identifiants des artistes, playlists, morceaux...car nous avons décidé de ne pas se baser dessus sur notre étude car c'est une suite de lettres et de chiffres. 
* Nous avons effectué quelques modifications sur les variables, en transformant certaines variables numériques en variables catégoriques et certains format d'affichage. 

Nous avons finalement souhaité étudier notre jeu de données initial en considérant deux jeux. Ce sont les suivants :
* Dans certains cas nous utilisons le **jeu de données initialement fourni** (avec les modification effectuées dessus).
* Dans d'autres cas nous utilisons un **jeu de données qui ne se base pas sur les playlists** auxquels les morceaux sont associés. Cela a permis de ne pas considérer les morceaux présents dans plusieurs playlists (identiques ou différentes) afin de pouvoir considérer uniquement leurs caractéristiques. De plus, afin de supprimer les éventuels *doublons* présents, nous avons fait le choix de conserver le morceau ayant la plus haute popularité. En effet, certains artistes sortent des EP et ces mêmes EPs sont également présents dans des albums, mais n'ont pas la même popularité. Donc pour ne pas biaiser d'éventuelles analyses, ce choix a été fait. 

# Analyse exploratoire
## Analyse univariée

In [ ]:
summary(onlysongs)

In [ ]:
#Affichage d'histogrammes représentant la distribution de la popularité des chansons, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_s, avec la possibilité de modifier la taille des titres et du texte des axes
options(repr.plot.width=15, repr.plot.height=15)
par(mfrow=c(4, 3))
hist(onlysongs$track_popularity, main="Popularité des chansons", xlab="Popularité", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$danceability, main="Danceability", xlab="Danceability", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$energy, main="Energy", xlab="Energy", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$loudness, main="Loudness", xlab="Loudness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$speechiness, main="Speechiness", xlab="Speechiness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$acousticness, main="Acousticness", xlab="Acousticness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$instrumentalness, main="Instrumentalness", xlab="Instrumentalness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$liveness, main="Liveness", xlab="Liveness", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$valence, main="Valence", xlab="Valence", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$tempo, main="Tempo", xlab="Tempo", ylab="Nombre de chansons", col="lightblue", border="black")
hist(onlysongs$duration_s, main="Durée des chansons", xlab="Durée (s)", ylab="Nombre de chansons", col="lightblue", border="black")

# Pour tempo et durée, on augmente le nombre de classes (breaks) pour avoir plus de barres
hist(onlysongs$tempo, main="Tempo", xlab="Tempo", ylab="Nombre de chansons", col="lightblue", border="black", breaks=40)
hist(onlysongs$duration_s, main="Durée des chansons", xlab="Durée (s)", ylab="Nombre de chansons", col="lightblue", border="black", breaks=40)


Nous pouvons constater que la majorité des caractéristiques représentées suivent une distribution qui semble normale.  

Concernant la distribution de la **popularité**, il y a une grande fréquence de morceaux dont la popularité est nulle. Cela s'explique par les critères de notations de *Spotify*, qui note la popularité d'un morceau en fonction de ses écoutes, si elles ont été nombreuses ou encore récentes.  

L'**acoustique** d'un morceau est noté entre 0.0 et 1.0. 
* 0.0 représente un morceau perçu comme non acoustique.
* 1.0 représente à l'inverse un morceau acoustique.  
  
Grâce à la distribution, nous pouvons constater qu'une grande fréquence des morceaux présents dans ce jeu possèdent une acoustique nulle, cela peut déjà donner une première analyse sur le fait que le jeu de données est principalement composé de musiques de type *électronique* ou *synthétique*. 

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
par(mfrow=c(2, 3))

# Histogramme pour la variable 'mode'
mode_counts <- table(onlysongs$mode)
barplot(
  mode_counts,
  main = "Distribution de la variable 'mode'",
  xlab = "Mode",
  ylab = "Fréquence",
  col = "lightblue",
  border = "black",
  las = 1 # Labels horizontaux
)

# Histogramme pour la variable 'key'
key_counts <- table(onlysongs$key)
barplot(
  key_counts,
  main = "Distribution de la variable 'key'",
  xlab = "Key",
  ylab = "Fréquence",
  col = "lightblue",
  border = "black",
  las = 1 # Labels horizontaux
)
# Conversion de la variable 'track_album_release_date' en format Date
onlysongs$track_album_release <- as.Date(onlysongs$track_album_release_date)

# Extraction de l'année de sortie
onlysongs$release_year <- format(onlysongs$track_album_release, "%Y")

# Comptage des occurrences par année
release_year_counts <- table(onlysongs$release_year)

# Création de l'histogramme
barplot(
  release_year_counts,
  main = "Distribution des années de sortie des albums",
  xlab = "Année de sortie",
  ylab = "Nombre de morceaux",
  col = "lightblue",
  border = "black",
  las = 2 # Rotation des labels pour les rendre lisibles
)

# Histogramme pour la variable 'playlist_genre'
genre_counts <- table(song$playlist_genre)
barplot(
  genre_counts,
  main = "Distribution de la variable 'playlist_genre'",
  xlab = "Genre de la playlist",
  ylab = "Fréquence",
  col = "lightblue",
  border = "black",
  las = 2 # Rotation des labels pour les rendre lisibles
)

# Histogramme pour la variable 'playlist_subgenre'
subgenre_counts <- table(song$playlist_subgenre)
barplot(
  subgenre_counts,
  main = "Distribution de la variable 'playlist_subgenre'",
  xlab = "Sous-genre de la playlist",
  ylab = "Fréquence",
  col = "lightblue",
  border = "black",
  las = 2 # Rotation des labels pour les rendre lisibles
)

Concernant la distribution des variables qualitatives, on remarque qu'au niveau des modes de musique, il y a une plus forte présence de gammes majeures que de gammes mineures. Pour le choix de la clé, on peut voir que la clé `D#E` est beaucoup moins présente que les autres. Pour ce qui est du reste des clés, on peut considérer qu'il n'y a pas vraiment d'autres extrêmes de présence ou d'absence. Pour la distribution des dates de sortie, nous avons un très gros pic de sortie d'albums en 2019. 
Finalement, pour la ditribution des genres et sous-genres, elle est à peu près similaires pour tous les genres, même si certains sous-genres sont beaucoup moins présents. Cela est dû à une présence de sous-genres moins connus.

In [ ]:
#Agrandis le texte des titres et des axes
options(repr.plot.width=15, repr.plot.height=10)

# Création des catégories pour la variable 'speechiness'
onlysongs$speechiness_category <- cut(
  onlysongs$speechiness,
  breaks = c(0, 0.33, 0.66, 1),
  labels = c("Music", "Speech and music", "Speech"),
  include.lowest = TRUE
)

# Comptage des occurrences dans chaque catégorie
speechiness_counts <- table(onlysongs$speechiness_category)

# Création de l'histogramme
barplot(
  speechiness_counts,
  main = "Distribution des catégories de 'speechiness'",
  xlab = "Catégories de speechiness",
  ylab = "Nombre de morceaux",
  col = "lightblue",
  border = "black"
)

# Création des catégories pour la variable 'instrumentalness'
onlysongs$instrumentalness_category <- cut(
  onlysongs$instrumentalness,
  breaks = c(0, 0.5, 1),
  labels = c("Vocal", "Instrumental"),
  include.lowest = TRUE
)

# Comptage des occurrences dans chaque catégorie
instrumentalness_counts <- table(onlysongs$instrumentalness_category)

# Création de l'histogramme
barplot(
  instrumentalness_counts,
  main = "Distribution des catégories de 'instrumentalness'",
  xlab = "Catégories de instrumentalness",
  ylab = "Nombre de morceaux",
  col = "lightblue",
  border = "black"
)

# Création des catégories pour la variable 'liveness'
onlysongs$liveness_category <- cut(
  onlysongs$liveness,
  breaks = c(0, 0.8, 1),
  labels = c("Studio", "Live"),
  include.lowest = TRUE
)

# Comptage des occurrences dans chaque catégorie
liveness_counts <- table(onlysongs$liveness_category)

# Création de l'histogramme
barplot(
  liveness_counts,
  main = "Distribution des catégories de 'liveness'",
  xlab = "Catégories de liveness",
  ylab = "Nombre de morceaux",
  col = "lightblue",
  border = "black"
)

Pour les variables speechiness, instrumentalness et liveness, les intervalles dans lesquels les morceaux ont été classés, s'interprètent de la manière suivante : 
* **speechiness :** 
  * $<0.33$ : faible présence de paroles
  * $[0.33,0.66]$ : présence modérée de paroles
  * $>0.66$ : forte présence de paroles 
* **instrumentalness :** 
  * $\le 0.5$ : peu ou pas instrumental
  * $>0.5$ : très instrumental
* **liveness :** 
  * $\le 0.8$ : morceau probablement produit en studio
  * $>0.8$ : morceau probablement produit en live (concert, public)

D'après les distributions obtenues, la majorité des morceaux sont considérés comme possédant peu de paroles et qui n'ont pas été produits en live. Concernant l'instrumentalité des morceaux, nous pouvons constater que la majorité des morceaux sont instrumentaux.

# Analyse multivariée

In [ ]:
# Affichage des colonnes de onlysongs
colnames(onlysongs)

# Affichage de la matrice de corrélation des colonnes popularity, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_s
correlation_matrix <- cor(onlysongs[, c("track_popularity", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_s")])
corrplot(correlation_matrix, method = "number")

Les variables semblent très peu corrélées entre elles pour la plupart, mais nous pouvons néanmoins constater les corrélations qui suivent.  
- **Corrélation positive :** 
    - *loudness* et *energy* : corrélation forte (0.69). Cela semble cohérent, on lie le dynamisme d'un morceau au bruit ressenti.
    - *valence* et *danceability* : légère corrélation (0.33). La valence mesure la positivité d'un morceau donc plus un morceau est positif, plus il est dansant.

- **Corrélation négative :**
    - *acousticness* et *energy* : corrélation négative modérée (-0.55). Plus une musique est perçue comme acoustique, moins elle est perçue comme énergique. Cela permet d'opposer des morceaux calmes à des morceaux dynamiques. 
    - *acousticness* et *loudness* : corrélation négative modérée (-0.38). Cela met en avant l'opposition entre un morceau bruyant et un morceau calme (acoustique).

Pour le reste on a des corrélations assez faibles, qu'elles soient positives ou négatives. Cela ne nous permet pas de mettre en avant d'autres liens entre les variables. 

In [ ]:
# Scatterplot pour visualiser la relation entre l'accousticness et l'energy
ggplot(onlysongs, aes(x = acousticness, y = energy)) +
  geom_point(aes(color = track_popularity), alpha = 0.5) +
  scale_color_gradient(low = "blue", high = "red") +
  labs(title = "Relation entre l'acousticness et l'energy",
       x = "Acousticness",
       y = "Energy",
       color = "Popularité") +
  theme_minimal()

In [ ]:
# Scatterplot pour visualiser la relation entre l'acousticness et la loudness
ggplot(onlysongs, aes(x = acousticness, y = loudness)) +
  geom_point(aes(color = track_popularity), alpha = 0.5) +
  scale_color_gradient(low = "blue", high = "red") +
  labs(title = "Relation entre l'acousticness et la loudness",
       x = "Acousticness",
       y = "Loudness",
       color = "Popularité") +
  theme_minimal()

In [ ]:
#Scatterplot pour visualiser la relation entre la loudness et l'energy
ggplot(onlysongs, aes(x = loudness, y = energy)) +
  geom_point(aes(color = track_popularity), alpha = 0.5) +
  scale_color_gradient(low = "blue", high = "red") +
  labs(title = "Relation entre la loudness et l'energy",
       x = "Loudness",
       y = "Energy",
       color = "Popularité") +
  theme_minimal()

In [ ]:
#scatterplot pour visualiser la relation entre la danceability et valence
ggplot(onlysongs, aes(x = danceability, y = valence)) +
  geom_point(aes(color = track_popularity), alpha = 0.5) +
  scale_color_gradient(low = "blue", high = "red") +
  labs(title = "Relation entre la danceability et la valence",
       x = "Danceability",
       y = "Valence",
       color = "Popularité") +
  theme_minimal()

Afin de vérifier ces corrélations, nous avons affiché les scatterplots les représentant.  

Les deux premiers graphiques représentent bien la **corrélation négative**. En effet lorsqu'une des deux variables évolue, la seconde diminue. En effet, entre les variables **Acousticness** contre **Energy et Loudness**, cette opposition vient du fait que les musiques acoustiques sont plus calmes, en opposition aux musiques énergétiques et bruyantes.

Pour les deux autres graphiques, à l'inverse nous remarquons bien des **corrélations positives**, en effet à mesure qu'une des deux variables augmente, la seconde aussi. Pour la **loudness et l'energy**, comme dit juste avant, une musique plus bruyante sera ressentie plus énergétique pour l'auditeur. Pour la **danceability et la valence**, une musique sur laquelle un auditeur voudra danser sera nécessairement plus joyeuse, ce qui est représenté par la valence.  

Nous avons décidé d'effectuer l'affichage en fonction de la **popularité**, mais comme nous pouvous le voir, il y a des morceaux de différentes popularités qui sont placés aux mêmes endroits, nous pouvons donc penser que la popularité n'a aucune influence sur les variables représentées.

In [ ]:
# Histogrammes de la popularité des chansons par genre, un histogramme par genre, en utilisant les données "song"
options(repr.plot.width=15, repr.plot.height=10)
par(mfrow=c(3, 2))
for (genre in levels(song$playlist_genre)) {
  hist(song$track_popularity[song$playlist_genre == genre], 
       main = paste("Popularité des chansons dans le genre", genre), 
       xlab = "Popularité", 
       ylab = "Nombre de chansons", 
       col = "lightblue", 
       border = "black")
}

In [ ]:
# Histogrammes de la popularité des chansons par sous-genre, un histogramme par sous-genre, en utilisant les données "song"
options(repr.plot.width=15, repr.plot.height=10)
par(mfrow=c(3, 2))
for (subgenre in levels(song$playlist_subgenre)) {
  hist(song$track_popularity[song$playlist_subgenre == subgenre], 
       main = paste("Popularité des chansons dans le sous-genre", subgenre), 
       xlab = "Popularité", 
       ylab = "Nombre de chansons", 
       col = "lightblue", 
       border = "black")
}

Pour la distribution des sous-genres, les distributions sont beaucoup plus hétérogènes. Effectivement, certains sous-genres étant moins présents que d'autres, la distribution de la popularité n'est pas la même que la distribution de la popularité globale.

In [ ]:
# Extraction de l'année de sortie
onlysongs$release_year <- format(onlysongs$track_album_release_date, "%Y")

# Calcul de la popularité moyenne par année de sortie
average_popularity_by_year <- onlysongs %>%
  group_by(release_year) %>%
  summarise(mean_popularity = mean(track_popularity, na.rm = TRUE))

# Création du graphique
ggplot(average_popularity_by_year, aes(x = as.numeric(release_year), y = mean_popularity)) +
  geom_line(color = "blue") +
  geom_point(color = "red") +
  labs(title = "Popularité moyenne des musiques par année de sortie",
       x = "Année de sortie",
       y = "Popularité moyenne") +
  theme_minimal()

In [ ]:
# Création d'une variable décennie à partir de l'année de sortie
onlysongs$decade <- paste0(substr(onlysongs$release_year, 1, 3), "0s")

# Calcul de la popularité moyenne par décennie
average_popularity_by_decade <- onlysongs %>%
    group_by(decade) %>%
    summarise(mean_popularity = mean(track_popularity, na.rm = TRUE))

# Tracé du graphique de popularité moyenne par décennie avec une courbe
ggplot(average_popularity_by_decade, aes(x = decade, y = mean_popularity, group = 1)) +
    geom_line(color = "blue", size = 1.2) +
    geom_point(color = "red", size = 3) +
    labs(title = "Popularité moyenne des musiques par décennie",
            x = "Décennie",
            y = "Popularité moyenne") +
    theme_minimal()


In [ ]:
# Comptage du nombre de morceaux par décennie
decade_counts <- onlysongs %>%
    group_by(decade) %>%
    summarise(
        n_morceaux = n(),
        pct_pop0 = mean(track_popularity == 0) * 100,
        pct_pop_note = mean(track_popularity > 0) * 100,
        sd_popularity = sd(track_popularity)
    ) %>%
    arrange(decade)

# Affichage du tableau
print(decade_counts)

**Par année :**
- Sur le premier graphe on peut lire la popularité moyenne des morceaux par année. Cela reste difficilement interprétable car il y a beaucoup de variations. 
- Nous pouvons néanmoins constater qu'avant 1960 il y a une tendance de popularité haute, mais qu'après ça oscille entre 30 et 50/60. 
- La moyenne de popularité totale est de 40.45.  

Pour une interprétation plus globale nous avons regardé la popularité des morceaux en les regroupant par décennie de parution.

**Par décennie :**
  - On a une tendance de popularité très haute pour les musiques sorties entre 1950-1980. Cela peut s'expliquer notamment pour les musiques sorties entre 1950-60 : 
      - Peu de musiques sorties pendant ces années sont présentes dans le jeu de données.
      - Il y a une moins grande diversité dans les notes comme il y a moins de musiques. Donc si celles-ci sont très bien notées alors ces années auront nécessairement une moyenne de popularité élevée.
      - Si nous croisons ces résultats à l'écart-type associé qui est élevé, nous pouvons conclure que par le faible nombre de morceaux, cette décennie n'est pas représentative. Les décennies suivantes sont plus stables. 
  - Le minimum des moyennes de popularité par décennie est atteint pour les musiques des années 2000 :
      - Cela peut s'expliquer par le nombre de morceaux qui n'ont pas de note de popularité, avec presque 19% des morceaux qui ont une note de 0. 
      - Donc soit ces morceaux n'ont été que très peu écoutés, soit les morceaux n'ont pas connu de succès, augmentant l'écart-type par rapport aux années précédentes. 
  - À l'inverse, dès 2020, la dispersion est plus faible avec peu de morceaux non notés, mais un nombre de morceaux assez faible, donc les morceaux ont une popularité en moyenne assez similaire avec moins de morceaux à succès et non succès extrêmes. 


In [ ]:
# Comptage du nombre de musiques par genre et par année
songs_by_genre_year <- song %>%
    group_by(release_year = format(track_album_release_date, "%Y"), playlist_genre) %>%
    summarise(count = n(), .groups = "drop")

# Création du graphique
    ggplot(songs_by_genre_year, aes(x = as.numeric(release_year), y = count, color = playlist_genre)) +
        geom_line() +
        labs(title = "Nombre de musiques sorties par genre et par année",
             x = "Année de sortie",
             y = "Nombre de morceaux",
             color = "Genre") +
        theme_minimal()

Nous avons observé les années de sortie en fonction du genre de musique. On remarque que le **rock** est le style qui était le plus présent entre les **années 1970 jusqu'à 1990**. Ensuite, les autres styles se sont développés très rapidement, notamment avec le style **EDM* qui est le plus présent dans les années **2010 à 2020**.

In [ ]:
# Calcul du nombre de playlists différentes par genre
playlists_per_genre <- song %>%
    group_by(playlist_genre) %>%
    summarise(n_playlists = n_distinct(playlist_name))

# Affichage du résultat
print(playlists_per_genre)

Nous avons effectué une rapide analyse univariée sur les genres et les sous-genres des playlists, donc sur le jeu de donné fourni initialement.   

Nous pouvons remarquer que le nombre de playlists par genre est plutôt équilibré avec en moyenne 76 playlists par genres. Néanmoins, on peut constater que le genre **EDM** est majoritairement présent dans le jeu de données, et que le genre **rock** est celui qui est minoritaire. Le reste des genres se différencie uniquement par quelques centaines de morceaux.  

In [ ]:
# Liste des variables numériques à tracer
numeric_vars <- c("track_popularity", "danceability", "energy", "loudness", "speechiness", 
                  "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_s")

# Affichage des courbes de densité pour chaque variable numérique selon le genre
library(ggplot2)
library(tidyr)

# Passage au format long pour ggplot2
song_long <- song %>%
  pivot_longer(cols = all_of(numeric_vars), names_to = "variable", values_to = "value")

# Tracé des courbes de densité pour chaque variable, colorées par genre
ggplot(song_long, aes(x = value, color = playlist_genre, fill = playlist_genre)) +
  geom_density(alpha = 0.2) +
  facet_wrap(~ variable, scales = "free", ncol = 3) +
  labs(title = "Distribution des variables numériques selon le genre de musique",
       x = "Valeur", y = "Densité", color = "Genre", fill = "Genre") +
  theme_minimal()

Afin d'essayer de comprendre les spécificités des morceaux présents dans le jeu de données, nous avons analysé les différentes **distributions des variables quantitatives** selon le genre des playlists. L'objectif est d'identifier ici les différences pouvant caractériser les genres musicaux. Ces courbes de densité permettent de visualiser où se concentrent la majorité des morceaux.  

<u>*Analyse des distributions et des genres :*</u>

* **Danceability :**
  * Le genre **latin** et le **rap** se distinguent particulièrement avec des valeurs de danceability proche de 0.8, indiquant des morceaux très dansants pour ces genres. 
  * À l'inverse, le genre **rock** est plus dispersé, indiquant que certains morceaux sont très dansants, d'autres pas du tout.
  
* **EDM**
  * Ce genre semble se distinguer sur plusieurs points, en effet d'après les distributions, les morceaux du type EDM auront tendance à donner un ressenti **dynamique (energy)** (densité autour de 0.9), **bruyant** et se dinstingue par son **tempo** se trouvant autour de 130 BPM, valeur caractérisant généralement les morceaux d'EDM, techno...
  * Néanmoins, les musiques de ce genre musical ne semblent pas être ressenties comme positives ou joyeuses d'après leur valence.
  * Et on retrouve bien la corrélation négative avec l'**acousticness**.
  
* **Popularity :**
  * Si l'on ne prend en compte le pic à 0 (qui concerne les morceaux non notés), mais seulement au second pic, on peut remarquer :
    * **EDM** semble être le genre le moins populaire, mais la présence de la queue de distribution vers les valeurs élevées peut traduire la présence de morceaux à succès. 
    * Le **rap** possède une popularité plutôt moyenne par rapport aux autres genres, et ne semble pas avoir beaucoup de morceaux ayant connu un grand succès, mais l'inverse est aussi vrai, il ne semble pas y avoir beaucoup de morceaux à faible audience.
    * La **pop** et **latin** sont les genres les plus populaires.

* **Energy :**
  * L'**EDM** est le genre le plus dynamique. 
  * La distribution plus large pour les genres comme **r&b**, **rap** ou encore **latin** aura plutôt tendance à indiquer que ces genres regroupent à la fois des morceaux calmes et énergiques.

* **Valence :**
  * Le genre **latin** se distingue particulièrement par sa valence élevée, traduisant des musiques plutôt joyeuses.
  * La distribution globale de la valence est large (tous genres confondus) traduisant une grande diversité émotionnelle produite par les morceaux des différents genres. 

In [ ]:
# Calcul des moyennes par genre pour chaque variable
average_values_by_genre <- song %>%
    group_by(playlist_genre) %>%
    summarise(
        danceability = mean(danceability, na.rm = TRUE),
        energy = mean(energy, na.rm = TRUE),
        loudness = mean(loudness, na.rm = TRUE),
        mode = mean(as.numeric(mode), na.rm = TRUE),
        speechiness = mean(speechiness, na.rm = TRUE),
        acousticness = mean(acousticness, na.rm = TRUE),
        instrumentalness = mean(instrumentalness, na.rm = TRUE),
        liveness = mean(liveness, na.rm = TRUE),
        valence = mean(valence, na.rm = TRUE),
        tempo = mean(tempo, na.rm = TRUE),
        duration_ms = mean(duration_s * 1000, na.rm = TRUE)
    )

# Affichage du tableau
print(average_values_by_genre)

Les données révèlent que les genres **EDM et Latin** se caractérisent par une énergie élevée et une forte propension à être dansant, ce qui les rend adaptés à des contextes festifs. La **pop** présente un équilibre entre **énergie** et **dansabilité**, tandis que le **R&B** et le **rap** mettent davantage l'accent sur les éléments parlés, avec une **énergie** modérée. Le **rock**, bien qu'énergique, est moins orienté vers la danse. Enfin, l'**EDM** est le genre le plus électronique, tandis que le **R&B** se distingue par son caractère plus acoustique. Ces caractéristiques reflètent les traits distinctifs propres à chaque genre musical.

In [ ]:
# Diagramme en barres empliées de la variable "Key" en fonction du genre de la musique
ggplot(song, aes(x = playlist_genre, fill = key)) +
  geom_bar(position = "fill") +
  labs(title = "Répartition de la variable 'Key' par genre de musique",
       x = "Genre de musique",
       y = "Proportion",
       fill = "Key") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

Dans cette distribution des clés par genre, nous ne remarquons pas de clés spécifiques à certains genres. Certaines clés sont plus présentes que d'autres dans les genres, mais rien de très significatif.

In [ ]:
#Diagramme en barre empillés de la variable "Mode" en fonction du genre de la musique
ggplot(song, aes(x = playlist_genre, fill = mode)) +
  geom_bar(position = "fill") +
  labs(title = "Répartition de la variable 'Mode' par genre de musique",
       x = "Genre de musique",
       y = "Proportion",
       fill = "Mode") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

Pour les modes en revanche, on voit que pour le **rock**, nous avons une tendance à avoir un mode plutôt **majeur**. Pour les autres styles de musiques, la répartition du mode est relativement équilibré, à 50-50.

In [ ]:
# Calculer le nombre de playlists différentes dans lesquelles chaque chanson apparaît
playlist_count <- song %>%
    group_by(track_name, track_artist) %>%
    summarise(n_playlists = n_distinct(playlist_name),
                        track_popularity = max(track_popularity)) %>%
    ungroup()

# Calcul de la corrélation entre le nombre de playlists et la popularité
correlation <- cor(playlist_count$n_playlists, playlist_count$track_popularity)

# Affichage du résultat
cat("Corrélation entre le nombre de playlists et la popularité :", correlation, "\n")

# Visualisation
ggplot(playlist_count, aes(x = n_playlists, y = track_popularity)) +
    geom_point(alpha = 1, color = "blue", size = 4) +
    labs(title = "Corrélation entre le nombre de playlists et la popularité",
             x = "Nombre de playlists",
             y = "Popularité de la chanson") +
    theme_minimal()

Finalement, nous avons décidé de regarder s'il y avait un lien entre la popularité d'une musique et le nombre de fois où elle est présente dans une playlist. On remarque que les musiques qui sont présentes dans beaucoup de playlists ont tendance à avoir une popularité élevée. En revanche, l'inverse n'est pas forcément vraie. En effet, des musiques avec une popularité élevée ($>80$) peuvent être présentes dans une unique playlist. Ce critère n'est donc pas forcément représentatif de la popularité d'une musique.

# Réduction de dimension

Dans cette partie, nous allons effectuer une analyse en composantes principales (ACP) sur les données prétraitées. L'ACP est une technique de réduction de dimension qui permet de projeter les données d'origine dans un espace de dimension inférieure. Nous avons gardé 20 variables et nous allons étudier s'il est possible de réduire la dimensionnalité de ces données tout en préservant un maximum d'information.

## Analyse en Composantes Principales (ACP)

Ici on ne sélectionne que les variables quantitatives, en y ajoutant une variable qualitative (playlist_genre) pour voir si elle a un impact sur la projection des données. On garde au final 11 variables quantitatives et 1 variable qualitative.

On décide de normaliser les données avant de faire l'ACP car l'ACP est sensible à l'échelle des variables. On va donc centrer et réduire les données.

In [ ]:
# PCA analysis using FactoMineR
song_pca <- song[, c(3, 7, 9:10, 12, 14:20)]

# Perform PCA
pca <- PCA(song_pca,scale.unit = TRUE, graph = FALSE,ncp = 7,quali.sup = 2)

# Afficher le pourcentage de variance expliquée par chaque composante principale
fviz_eig(pca, addlabels = TRUE, ylim = c(0, 40))

# Calculer la variance cumulée
explained_variance <- pca$eig[, 2]  # La deuxième colonne contient le pourcentage de variance expliquée
cumulative_variance <- cumsum(explained_variance) 

# Tracer la variance cumulée
plot(cumulative_variance, xlab = "Nombre de composantes principales", ylab = "Variance cumulée", type = "b")
abline(h = 80, col = "red", lty = 2)  # Ligne horizontale à 80% de variance expliquée

**Interprétation :** 

L’analyse de la variance expliquée montre que les 7 premières composantes principales permettent de représenter 80,1 % de la variance totale du jeu de données. Cela signifie que l’essentiel de l’information contenue dans les 11 variables numériques initiales (comme danceability, energy, speechiness, tempo, etc.) peut être résumé avec seulement 7 dimensions, ce qui représente une réduction significative de la complexité du dataset tout en conservant une bonne qualité descriptive.

Dans cette optique de réduction de dimension, il serait pertinent de conserver ces 7 composantes principales pour la suite des analyses (clustering, visualisation, classification), car elles capturent la structure principale des données tout en éliminant le "bruit".

Dans l’analyse factorielle, nous avons choisi d’interpréter les trois premières composantes principales, qui à elles seules expliquent 45 % de la variance totale. Elles offrent un bon compromis entre lisibilité et pertinence pour une visualisation ou une première analyse des relations entre les variables et les genres musicaux (playlist_genre).

In [ ]:
# Corrélation des variables
corrplot(pca$var$cor, is.corr = FALSE, method = "ellipse")

# Tracer les variables sur le plan factoriel dim 1-2
fviz_pca_var(pca, axes=c(1,2),col.var = "contrib", gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), repel = TRUE) +
  labs(title = "Variables sur le plan factoriel") +
  theme_minimal()

# Tracer les variables sur le plan factoriel dim 1-3
  fviz_pca_var(pca, axes = c(1, 3), col.var = "contrib", 
         gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
         repel = TRUE) +
    labs(title = "Variables sur le plan factoriel (Dim 1-3)") +
    theme_minimal()

# Tracer les variables sur le plan factoriel dim 2-3
  fviz_pca_var(pca, axes = c(2, 3), col.var = "contrib", 
         gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"), 
         repel = TRUE) +
    labs(title = "Variables sur le plan factoriel (Dim 2-3)") +
    theme_minimal()

La table des correlations et les trois graphiques ci-dessus représentent les projections des features sur les trois premières composantes principales nous donne des informations sur la structure des données réduites.

**Composante principale 1 :**
    
Les variables **energy** (-0.91), **loudness** (-0.80) et **acousticness** (+0.72) sont linéairement corrélées avec la première composante principale, en soulignant que energy et loudness sont inversément corrélées avec acousticness. Cela indique que la CP1 oppose les morceaux énergiques, forts en volume et peu acoustiques (ex : rock, électro) aux morceaux calmes, acoustiques et peu énergétiques (ex : folk, classique).

**Composante principale 2 :**

Sur le graphique de gauche, on remarque une opposition des variables **instrumentalness** (+0.45), **duration_s** (+0.38) contre **danceability** (-0.68), **valence** (-0.62), **track_popularity** (-0.37), **speechiness** (-0.39).

Cette composante principale oppose deux profils de morceaux :
    D’un côté, les morceaux instrumentaux, longs et peu populaires (forte contribution de instrumentalness et duration_s), souvent associés à des genres comme le classique ou le jazz.
    De l’autre, les chansons courtes, dansantes, joyeuses et populaires (forte contribution de danceability, valence et track_popularity), typiques de la pop ou de la musique de club.
    Enfin, cette opposition suggère que les morceaux avec des paroles marquées (speechiness) et une structure rythmique engageante (danceability) sont plus susceptibles de générer de la popularité.

**Composantes principales 2 et 3 :**

Sur le graphique au centre, on peut extraire plusieurs informations :
    Les morceaux dansants et joyeux (haute valence) s'opposent dans une moindre mesure aux morceaux de faible tempo. De plus, ces morceaux semblent avoir peu de versions live.
    On observe aussi que les morceaux instrumentaux et longs ont généralement une faible popularité, tandis que les morceaux courts et peu instrumentaux sont souvent plus populaires, ce qui est typique de la musique pop, qui est souvent plus accessible et commerciale.

**Composante principale 3 :**

La troisième composante (CP3) révèle un paradoxe : elle regroupe des morceaux à fort potentiel dansant (danceability) et mood positif (valence), mais qui restent peu populaires (track_popularity). Ces morceaux sont souvent instrumentaux (instrumentalness), longs (duration_s) et à tempo faible (tempo), ce qui les éloigne des standards des charts. Cette composante pourrait représenter des créations artistiques équilibrant danse et complexité, mais peinant à atteindre un large public.


La troisième composante principale met en lumière une tension plus subtile entre deux types de morceaux aux caractéristiques inattendues.

Du côté des contributions négatives, on retrouve des titres de pop et R&B calmes et acoustiques comme raindrops (an angel cried) (Ariana Grande) ou You Are The Reason (Calum Scott). Ces chansons ont :

    une acousticness élevée (acapela, guitare acoustique, piano),
    un tempo légèrement plus rapide que la médiane des morceaux,
    une faible énergie, mais un potentiel émotionnel fort (valence variable).

Comme suggéré par la PC3, ces morceaux rencontrent un certain succès, illustrant un profil de chansons émotionnelles, accessibles et bien produites, souvent interprétées par des artistes grand public au style sobre et expressif.

Les contributions positives, quant à elles, sont largement dominées par des morceaux EDM ou latino instrumentaux, comme I Feel Love ou Chase. Ces morceaux sont :

    longs,
    très instrumentaux,
    énergiques mais souvent moins "accessibles" émotionnellement (valence très élevée mais peu de paroles, structure répétitive).

Ils présentent également une popularité extrêmement faible, allant de 0 à 8. Ces productions s’adressent probablement à un public averti ou sont conçues pour des usages spécifiques (DJ sets, ambiances electro), ce qui les éloigne des standards de la musique grand public.

Ces observations confirment que la PC3 oppose des créations acoustiques à forte charge émotionnelle à des morceaux instrumentaux, électroniques, longs, aux dynamiques parfois complexes ou répétitives.

On remarque le signe des contributions est inversé par rapport à l'analyse en composantes principales (ACP) réalisée sous Python.

**Remarque:** on peut noter que les valeurs ne sont pas exactement les mêmes que sur le notebook Python. L'ACP sous R ne prend pas forcément la même base que sur Python, ce qui explique les valeurs parfois négatives ou positives.

Pour mieux comprendre à quoi correspond les composantes, nous allons regarder les morceaux (individus) contribuant le plus à chacune des composantes principales.

In [ ]:
# Extraction des coordonnées des individus sur la dimension 1
dim1_coords <- pca$ind$coord[, 1]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices <- order(dim1_coords)[1:5]
max_indices <- order(dim1_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals <- song[c(min_indices, max_indices), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals$Dim1_Value <- dim1_coords[c(min_indices, max_indices)]

# Réorganiser les colonnes pour mettre Category et Dim1_Value au début
col_order <- c("Category", "Dim1_Value", colnames(extreme_individuals)[1:(ncol(extreme_individuals)-2)])
extreme_individuals <- extreme_individuals[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals)
}



Pour mieux cerner les types de morceaux représentés aux extrémités de la première composante principale (PC1), nous avons identifié les individus (chansons) ayant les contributions les plus élevées, positives comme négatives.

Du côté des **contributions positives**, on retrouve majoritairement des morceaux **rock, hard rock ou pop rock très énergiques et puissants** tels que American Idiot (Green Day), Beauty Queen (BLVK SWVN) ou ATTENTION ATTENTION (Shinedown). Ces morceaux sont caractérisés par une énergie élevée, une forte intensité sonore (loudness) et une faible acoustique, ce qui confirme bien la structure mise en évidence par la CP1. Notons aussi This Is How We Do It (Montell Jordan), un morceau R&B énergique, qui se distingue des autres par son genre mais partage les mêmes caractéristiques acoustiques.

À l’opposé, les morceaux à **contribution très négative** sur PC1 sont des titres à **forte acoustique, peu énergiques et très faibles en loudness**. Il s'agit notamment de sons ambiants, relaxants ou naturels, comme Peaceful Forest ou Tropical Rainforest at Dawn, mais aussi de titres R&B ou indie très doux (Small de chloe moriondo). Ces morceaux incarnent l'autre extrémité de la CP1 : des chansons calmes, acoustiques et à faible énergie, souvent issues de sous-genres comme tropical, indie poptimism ou new jack swing.

Cette opposition renforce l’interprétation de la CP1 comme un axe énergie / intensité sonore vs. calme / acoustique, pertinent pour distinguer deux grandes familles de styles musicaux dans le dataset.

Nous faison de même pour les deux autres composantes principales, en regardant les morceaux qui contribuent le plus à chacune des composantes.


In [ ]:
# Extraction des coordonnées des individus sur la dimension 2
dim2_coords <- pca$ind$coord[, 2]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices_dim2 <- order(dim2_coords)[1:5]
max_indices_dim2 <- order(dim2_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals_dim2 <- song[c(min_indices_dim2, max_indices_dim2), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals_dim2$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals_dim2$Dim2_Value <- dim2_coords[c(min_indices_dim2, max_indices_dim2)]

# Réorganiser les colonnes pour mettre Category et Dim2_Value au début
col_order <- c("Category", "Dim2_Value", colnames(extreme_individuals_dim2)[1:(ncol(extreme_individuals_dim2)-2)])
extreme_individuals_dim2 <- extreme_individuals_dim2[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals_dim2)
}



Côté contributions positives, on retrouve des titres principalement rap et latino, tels que Suge de DaBaby ou LAX de B0nds. Ces morceaux sont :

    courts,
    dansants (haute danceability),
    avec une valence élevée (émotion positive),
    mais également avec un certain niveau de speechiness, notamment pour les titres rap.

Ces morceaux partagent donc des caractéristiques propres aux chansons énergétiques, rythmées et populaires, souvent taillées pour le streaming, avec des formats courts, accrocheurs et directs.

À l’opposé, les morceaux ayant une forte contribution négative à PC2 sont très différents : on retrouve des paysages sonores naturels, ambiants ou instrumentaux comme Rain Forest and Tropical Beach Sound, Caribbean Thunderstorm, ou encore Battlement. Ces titres sont :

    longs,
    instrumentaux (forte instrumentalness),
    avec une faible valence et peu de parole,
    et souvent issus de sous-genres comme tropical, album rock, ou ambient.

Cela confirme l’interprétation initiale de la PC2 comme un axe opposant la musique instrumentale, longue et contemplative à une musique populaire, dansante et rythmée.


In [ ]:
# Extraction des coordonnées des individus sur la dimension 3
dim3_coords <- pca$ind$coord[, 3]

# Récupération des indices des 5 valeurs minimales et maximales
min_indices_dim3 <- order(dim3_coords)[1:5]
max_indices_dim3 <- order(dim3_coords, decreasing = TRUE)[1:5]

# Sélection complète des individus extrêmes avec toutes leurs caractéristiques
extreme_individuals_dim3 <- song[c(min_indices_dim3, max_indices_dim3), ]

# Ajout d'une colonne pour identifier la catégorie (minimum ou maximum)
extreme_individuals_dim3$Category <- c(rep("Minimum", 5), rep("Maximum", 5))
extreme_individuals_dim3$Dim3_Value <- dim3_coords[c(min_indices_dim3, max_indices_dim3)]

# Réorganiser les colonnes pour mettre Category et Dim3_Value au début
col_order <- c("Category", "Dim3_Value", colnames(extreme_individuals_dim3)[1:(ncol(extreme_individuals_dim3)-2)])
extreme_individuals_dim3 <- extreme_individuals_dim3[, col_order]

# Pour une meilleure visualisation en format tableau
if (requireNamespace("knitr", quietly = TRUE)) {
  knitr::kable(extreme_individuals_dim3)
}

La troisième composante principale met en lumière une tension plus subtile entre deux types de morceaux aux caractéristiques inattendues.

Du côté des contributions négatives, on retrouve des titres de pop et R&B calmes et acoustiques comme raindrops (an angel cried) (Ariana Grande) ou You Are The Reason (Calum Scott). Ces chansons ont :

    une acousticness élevée (acapela, guitare acoustique, piano),
    un tempo légèrement plus rapide que la médiane des morceaux,
    une faible énergie, mais un potentiel émotionnel fort (valence variable).

Comme suggéré par la PC3, ces morceaux rencontrent un certain succès, illustrant un profil de chansons émotionnelles, accessibles et bien produites, souvent interprétées par des artistes grand public au style sobre et expressif.

Les contributions positives, quant à elles, sont largement dominées par des morceaux EDM ou latino instrumentaux, comme I Feel Love ou Chase. Ces morceaux sont :

    longs,
    très instrumentaux,
    énergiques mais souvent moins "accessibles" émotionnellement (valence très élevée mais peu de paroles, structure répétitive).

Ils présentent également une popularité extrêmement faible, allant de 0 à 8. Ces productions s’adressent probablement à un public averti ou sont conçues pour des usages spécifiques (DJ sets, ambiances electro), ce qui les éloigne des standards de la musique grand public.

Ces observations confirment que la PC3 oppose des créations acoustiques à forte charge émotionnelle à des morceaux instrumentaux, électroniques, longs, aux dynamiques parfois complexes ou répétitives.

On remarque le signe des contributions est inversé par rapport à l'analyse en composantes principales (ACP) réalisée sous Python.

## MDS

Le **MDS (Multidimensional Scaling)** est une technique de réduction de dimension non linéaire qui permet de visualiser des données en préservant les distances entre les points. Dans notre cas, nous allons l'appliquer sur les données prétraitées pour obtenir une représentation en 2D des morceaux.

In [ ]:
# 1. Sélectionner les variables numériques pertinentes (similaire à l'ACP)
numerical_features <- song[, c("danceability", "energy", "loudness", "speechiness", 
                               "acousticness", "instrumentalness", "liveness", 
                               "valence", "tempo", "track_popularity", "duration_s")]

# S'assurer qu'il n'y a pas de NA
numerical_features <- na.omit(numerical_features)

# Limiter à 2000 chansons pour éviter crash mémoire
set.seed(42)
if (nrow(numerical_features) > 2000) {
  sample_idx <- sample(seq_len(nrow(numerical_features)), 2000)
  numerical_features <- numerical_features[sample_idx, ]
}

# Standardiser
scaled_numerical_features <- scale(numerical_features)
dist_matrix <- dist(scaled_numerical_features)

# MDS sans add=TRUE
mds_result <- cmdscale(dist_matrix, k = 2, eig = TRUE)

mds_points <- as.data.frame(mds_result$points)
colnames(mds_points) <- c("Dim1", "Dim2")

# Récupérer le genre correspondant
if (exists("sample_idx")) {
  temp_df_for_mds <- song[, c("playlist_genre", "danceability", "energy", "loudness", "speechiness", 
                              "acousticness", "instrumentalness", "liveness", 
                              "valence", "tempo", "track_popularity", "duration_s")]
  temp_df_for_mds_clean <- na.omit(temp_df_for_mds)
  temp_df_for_mds_clean <- temp_df_for_mds_clean[sample_idx, ]
} else {
  temp_df_for_mds <- song[, c("playlist_genre", "danceability", "energy", "loudness", "speechiness", 
                              "acousticness", "instrumentalness", "liveness", 
                              "valence", "tempo", "track_popularity", "duration_s")]
  temp_df_for_mds_clean <- na.omit(temp_df_for_mds)
}

mds_points$playlist_genre <- temp_df_for_mds_clean$playlist_genre

# Visualisation
ggplot(mds_points, aes(x = Dim1, y = Dim2, color = playlist_genre)) +
  geom_point(alpha = 0.7) +
  labs(title = "MDS des chansons (basé sur les caractéristiques audio)",
       x = "Dimension MDS 1",
       y = "Dimension MDS 2",
       color = "Genre de Playlist") +
  theme_minimal() +
  guides(color = guide_legend(override.aes = list(alpha = 1, size = 3)))

# Goodness-of-fit
cat("\nGoodness-of-fit (GOF):\n")
print(mds_result$GOF)


Nous allons désormais réaliser un **t-SNE** (t-distributed Stochastic Neighbor Embedding) pour visualiser en 2D la proximité (ou la similarité) entre les chansons Spotify selon leurs caractéristiques audio (danceability, energy, loudness, etc.), pour voir **si des genres musicaux distincts émergent sous forme de groupes**. A l'inverse de la PCA et de la MDS, le **t-SNE est une méthode non linéaire** qui pourrait capturer des relations différentes entre les données que celles révélées par les méthodes linéaires.

## t-SNE

In [ ]:
install.packages("Rtsne")
library(Rtsne)
library(ggplot2)

# Supprimer les doublons
scaled_numerical_features_unique <- unique(scaled_numerical_features)

# Identifier les lignes uniques
unique_rows <- !duplicated(scaled_numerical_features)

# Conserver uniquement les genres correspondant aux lignes uniques
playlist_genre_unique <- temp_df_for_mds_clean$playlist_genre[unique_rows]

# Exécuter t-SNE
set.seed(42)
tsne_result <- Rtsne(scaled_numerical_features_unique, dims = 2, perplexity = 30, verbose = TRUE)

# Préparer le dataframe pour ggplot
tsne_df <- as.data.frame(tsne_result$Y)
colnames(tsne_df) <- c("Dim1", "Dim2")
tsne_df$playlist_genre <- playlist_genre_unique

# Visualisation
ggplot(tsne_df, aes(x = Dim1, y = Dim2, color = playlist_genre)) +
  geom_point(alpha = 0.6) +
  labs(title = "t-SNE des chansons (caractéristiques audio)",
       x = "Dimension 1", y = "Dimension 2",
       color = "Genre de Playlist") +
  theme_minimal() +
  guides(color = guide_legend(override.aes = list(size = 4, alpha = 1)))


## Interprétation des méthodes de MDS et t-SNE

### Interprétation du MDS

Le graphique issu de la MDS montre une concentration importante des points au centre, avec un fort chevauchement entre les différents genres musicaux (`pop`, `rap`, `r&b`, `rock`, etc.). Il n'y a pas de séparation nette ou de regroupement clair visible sur les deux dimensions principales.

Le **Goodness-of-Fit (GOF)** est d’environ **0.34**, ce qui signifie que seulement **34 %** de la structure initiale des distances est préservée dans cette projection bidimensionnelle. Ce score relativement faible traduit une **perte d'information importante**, ce qui rend la visualisation difficile à interpréter de manière fiable.

Cela suggère que :

- Les genres musicaux ne se distinguent pas clairement sur la base des distances euclidiennes entre leurs caractéristiques audio normalisées.
- La MDS, étant une méthode **linéaire**, ne capture pas bien les relations **non linéaires** qui pourraient exister entre les chansons.

---

### Interprétation du t-SNE

Le graphique t-SNE montre une plus grande dispersion des points, avec des zones où certains genres comme `edm` ou `rap` semblent former des **sous-groupes partiellement distincts**. Cependant, on observe encore un chevauchement significatif entre les genres.

Contrairement à la MDS, le t-SNE est une méthode **non linéaire** qui vise à préserver les **voisinages locaux** plutôt que les distances globales. Cela lui permet de mieux mettre en évidence les structures locales, comme des groupes compacts, même si les distances globales ne sont pas interprétables.

L’interprétation du t-SNE suggère que :

- Il existe quelques **régions homogènes** selon certains genres, mais aucune **séparation franche** entre clusters de genres.
- Les caractéristiques audio seules ne suffisent probablement pas à **discriminer clairement** les genres musicaux.
- Le t-SNE offre une visualisation plus riche que la MDS, mais reste difficile à interpréter en l’absence de clusters bien définis.

---

### Conclusion commune

Les deux méthodes révèlent que les genres musicaux dans ce jeu de données **ne sont pas linéairement séparables** sur la base des caractéristiques audio fournies. Le chevauchement visuel suggère soit :

- une **proximité réelle** entre les genres sur le plan acoustique,
- soit un **manque de variabilité** ou de pertinence dans les variables utilisées.

## Analyse en Correspondances Multiples (MCA)

### Objectif

L'Analyse en Correspondances Multiples (MCA) va être utilisée afin de visualiser les associations entre les modalités des variables qualitatives et d'identifier des clusters ou des oppositions significatives.

### Transformation des variables quantitatives en catégories

Pour enrichir l'analyse des correspondances multiples (MCA), nous proposons de convertir les variables quantitatives en variables qualitatives ordinales. Cette transformation permet d'intégrer les caractéristiques audio numériques dans une analyse factorielle adaptée aux données catégorielles.

#### Critères de segmentation

La catégorisation tient compte des seuils naturels d'interprétation des caractéristiques audio :

**Variables binaires (seuil à 0.5)** : 
- `danceability`, `energy`, `valence` : Séparation entre niveaux faible/élevé selon la médiane théorique
- `instrumentalness` : Distinction claire entre morceaux vocaux et instrumentaux

**Variables à seuils spécifiques** :
- `speechiness` : Seuil à 0.3 pour différencier musique pure vs. contenu parlé (rap, podcast)
- `liveness` : Seuil à 0.8 pour capturer les vraies performances live
- `popularity` : Segmentation en tertiles (0-20, 20-75, 75-100) reflétant les distributions réelles de Spotify

**Variables temporelles** :
- `tempo` : Classification musicologique standard (lent <100, modéré 100-150, rapide >150 BPM)
- `duration` : Segmentation basée sur les formats musicaux (courts <2min, moyens 2-4min, longs >4min)
- `decade` : Regroupement par décennies avec fusion des années 1950s-1960s pour équilibrer les effectifs

Cette approche préserve la signification musicale des variables tout en créant des catégories équilibrées pour l'analyse MCA.

In [ ]:
# Création d'un data.frame catégorique selon les règles demandées

df_cat_custom <- data.frame(row.names = rownames(song))

# Popularity
df_cat_custom$popularity_cat <- cut(
    song$track_popularity,
    breaks = c(-1, 20, 75, 100),
    labels = c("Peu populaire", "Popularité moyenne", "Très populaire"),
    include.lowest = TRUE
)

# Speechiness
df_cat_custom$speechiness_cat <- cut(
    song$speechiness,
    breaks = c(-0.01, 0.3, 1.0),  # Ajuster les seuils
    labels = c("Peu de paroles", "Paroles dominantes"),  # Deux classes seulement
    include.lowest = TRUE
)

# Danceability
df_cat_custom$danceability_cat <- cut(
    song$danceability,
    breaks = c(-0.01, 0.5, 1.0),
    labels = c("Peu dansant", "Dansant"),
    include.lowest = TRUE
)

# Energy
df_cat_custom$energy_cat <- cut(
    song$energy,
    breaks = c(-0.01, 0.5, 1.0),
    labels = c("Peu énergique", "Energique"),
    include.lowest = TRUE
)

# Instrumentalness
df_cat_custom$instrumentalness_cat <- cut(
    song$instrumentalness,
    breaks = c(-0.01, 0.5, 1.0),
    labels = c("Peu instrumental", "Instrumentale"),
    include.lowest = TRUE
)

# Liveness
df_cat_custom$liveness_cat <- cut(
    song$liveness,
    breaks = c(-0.01, 0.8, 1.0),
    labels = c("Pas live", "Live"),
    include.lowest = TRUE
)

# Valence
df_cat_custom$valence_cat <- cut(
    song$valence,
    breaks = c(-0.01, 0.5, 1.0),
    labels = c("Triste", "Joyeux"),
    include.lowest = TRUE
)

# Tempo
df_cat_custom$tempo_cat <- cut(
    song$tempo,
    breaks = c(-Inf,100, 150, Inf),
    labels = c("Tempo lent", "Tempo modéré", "Tempo rapide"),
    include.lowest = TRUE
)

# Duration (en secondes)
df_cat_custom$duration_cat <- cut(
    song$duration_s,
    breaks = c(-0.01, 120, 240, Inf),
    labels = c("Morceaux courts", "Morceaux moyennement longs", "Morceaux longs"),
    include.lowest = TRUE
)

# Décennie de sortie de l'album
years <- as.numeric(format(song$track_album_release_date, "%Y"))
df_cat_custom$decade <- ifelse(
    !is.na(years),
    paste0(floor(years / 10) * 10, "s"),
    "unknown"
)

#Fusionner 50s et 60s
df_cat_custom$decade <- recode(df_cat_custom$decade, 
                               "1950s" = "1950s-1960s", 
                               "1960s" = "1950s-1960s")

head(df_cat_custom)

Dans un premier temps, nous allons nous intéresser aux trois variables catégorielles suivantes :Les variables qualitatives sélectionnées pour cette première analyse sont :
- `playlist_genre` : Genre de la playlist (ex. rock, pop, rap, etc.).
- `key` : Tonalité musicale (ex. C, D, E♭, etc.).
- `mode` : Mode musical (majeur ou mineur).

In [ ]:
# Sélection de toutes les variables qualitatives pertinentes pour la MCA
qual_vars <- c( 'playlist_genre', 'key', 'mode')
song_mca_all <- song[, qual_vars]

# Réalisation de la MCA avec FactoMineR
mca_all <- MCA(song_mca_all, graph = FALSE)

# Visualisation des modalités sur le plan factoriel
fviz_mca_var(mca_all, col.var = "cos2", 
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE) +
  labs(title = "MCA - Toutes les variables qualitatives") +
  theme_minimal()

### Résultats

#### 1. Axes factoriels
- **Dim1 (8%)** : Cet axe semble opposer des genres musicaux et des tonalités spécifiques :
  - À droite, des genres comme `rap` et des tonalités comme `A♯/B♭` sont associés à des morceaux modernes ou spécifiques.
  - À gauche, des tonalités comme `C` et `G` sont associées à des genres comme `rock`, suggérant une relation avec des styles plus classiques.
- **Dim2 (6.6%)** : Cet axe reflète une distinction entre les modes (`major` et `minor`) et leur association avec certains genres :
  - En haut, le mode `major` est associé à des genres comme `rock`.
  - En bas, le mode `minor` est plus proche de genres comme `rap` et `edm`.

#### 2. Proximité des modalités
- Les modalités proches sur le graphique sont souvent associées dans les données :
  - `pop`, `latin`, `r&b` et `edm` sont centrés par rapport aux modes `minor`et `major`, indiquant qu'ils n'appartiennent pas clairement à un mode spécifique, mais partagent des caractéristiques communes.
  - `rap` est également centré par rapport à ces modes, ce qui suggère que l'utilisation des modes majeurs et mineurs est assez équilibrée dans la composition des morceaux de rap. Toutefois, il reste distinct des groupes `pop`, `latin`, `r&b` et `edm`.
  - `rock` est plus proche de `major`, ce qui suggère que les musiques de ce genre sont souvent associées à des tonalités majeures.
  - Les tonalités comme `B`,`D♯/E♭`,`A♯/B♭`,`F♯/G♭`et `F` sont proches de `minor`, indiquant qu'elles sont souvent utilisées dans des morceaux en mode mineur.
  - Les tonalités comme `C`, `G`,`D` sont proches de `major`, ce qui suggère qu'elles sont souvent utilisées dans des morceaux en mode majeur.

#### 3. Cos2 (Qualité de représentation)
- Les couleurs des points indiquent la qualité de représentation des modalités sur les deux premières dimensions :
  - Les modalités avec des couleurs chaudes (orange/rouge) comme `major` ou `minor` sont bien représentées sur ces axes.
  - Les modalités avec des couleurs froides (bleu/vert) comme certaines tonalités (`C`, `G`) sont moins bien représentées, ce qui signifie qu'elles pourraient être mieux expliquées par d'autres dimensions.

### Conclusion
Cette MCA met en évidence des associations claires entre les genres musicaux, les tonalités (`key`), et les modes (`major`/`minor`). Elle permet de visualiser les relations qualitatives dans les données et d'identifier des clusters ou des oppositions significatives. Par exemple :
- `rock` est distinct des autres genres, avec des tonalités et un mode spécifiques.
- `pop`, `latin`, `r&b` et `edm` partagent des caractéristiques similaires, mais ne sont pas clairement associés à un mode particulier.
- `rap` est centré par rapport aux modes, mais reste distinct des autres genres.

Ces résultats offrent une meilleure compréhension des structures qualitatives des données et peuvent être utilisés pour des analyses complémentaires, comme la segmentation ou la classification.

On se propose ensuite d'identifier les top artists afin de valider cette MCA et de voir si les artistes les plus populaires sont bien représentés dans les genres identifiés.

In [ ]:
# Identifier les top artists
top_artists <- c("Eminem", "Green Day", "David Guetta", "Ed Sheeran")

# Créer une nouvelle variable track_artist_grouped avec "other" pour les artistes non sélectionnés
song$track_artist_grouped <- as.character(song$track_artist)
song$track_artist_grouped[!(song$track_artist %in% top_artists)] <- "other"
song$track_artist_grouped <- as.factor(song$track_artist_grouped)

# Filtrer le dataset pour ne garder que les morceaux des top artists ou "other"
song_top_other <- song[song$track_artist_grouped %in% c(top_artists, "other"), ]

# Sélection des variables qualitatives pour la MCA
qual_vars_top_other <- c('track_artist_grouped', 'playlist_genre', 'key', 'mode')
song_mca_top_other <- song_top_other[, qual_vars_top_other]

# Réalisation de la MCA
mca_top_other <- MCA(song_mca_top_other, graph = FALSE)

# Visualisation des modalités sur le plan factoriel
fviz_mca_var(mca_top_other, col.var = "cos2",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE) +
  labs(title = "MCA - Top 4 artistes vs autres + genre, key, mode") +
  theme_minimal()

### Résultats

#### 1. Proximité des modalités
- Les artistes proches sur le graphique partagent des caractéristiques similaires :
  - `Ed Sheeran` est proche des tonalités `A` et `D`, ainsi que du mode `major`, ce qui reflète son style musical souvent associé à des tonalités classiques.
  - `Green Day` est également associé au mode `major` et à des tonalités comme `C` et `D`, typiques de leur style rock.
  - `David Guetta` est lié à des tonalités comme `F` et `D♯/E♭`, souvent utilisées dans la musique électronique.
  - `Eminem` est associé au mode `minor` et à des tonalités comme `A♯/B♭`, caractéristiques de son style rap.

#### 2. Clusters identifiés
  - `rock` est fortement associé à `Green Day` et au mode `major`.
  - `rap` est lié à `Eminem` et au mode `minor`.
  - `Ed Sheeran` est proche du centre, indiquant un style musical équilibré entre tonalités et modes.

Dans un deuxième temps, nous allons effectuer une Analyse des Correspondances Multiples (MCA) en rajoutant les variables qualitatives construites à partir des variables quantitatives. Cela nous permettra d'explorer les relations entre les genres musicaux et les caractéristiques audio de manière plus approfondie.

#### Variables utilisées

Les variables qualitatives sélectionnées pour cette analyse sont :
- `playlist_genre` : Genre de la playlist (ex. rock, pop, rap, edm, r&b, latin).
- `popularity_cat` : Catégorie de popularité (Peu populaire, Popularité moyenne, Très populaire).
- `speechiness_cat` : Présence de paroles (Peu de paroles, Paroles dominantes).
- `danceability_cat` : Potentiel dansant (Peu dansant, Dansant).
- `energy_cat` : Niveau d'énergie (Peu énergique, Énergique).
- `instrumentalness_cat` : Caractère instrumental (Peu instrumental, Instrumentale).
- `liveness_cat` : Caractère live (Pas live, Live).
- `valence_cat` : Valence émotionnelle (Triste, Joyeux).
- `tempo_cat` : Catégorie de tempo (Tempo lent, Tempo modéré, Tempo rapide).
- `duration_cat` : Durée des morceaux (Morceaux courts, Morceaux moyennement longs, Morceaux longs).
- `decade` : Décennie de sortie (1950s-1960s, 1970s, 1980s, 1990s, 2000s, 2010s, 2020s).
---

In [ ]:
# D'abord, ajouter les variables catégorielles créées précédemment à song
song$popularity_cat <- df_cat_custom$popularity_cat
song$speechiness_cat <- df_cat_custom$speechiness_cat
song$danceability_cat <- df_cat_custom$danceability_cat
song$energy_cat <- df_cat_custom$energy_cat
song$instrumentalness_cat <- df_cat_custom$instrumentalness_cat
song$liveness_cat <- df_cat_custom$liveness_cat
song$valence_cat <- df_cat_custom$valence_cat
song$tempo_cat <- df_cat_custom$tempo_cat
song$duration_cat <- df_cat_custom$duration_cat
song$decade <- df_cat_custom$decade

# MCA avec toutes les variables qualitatives + les variables catégorielles créées
qual_vars_cat <- c('playlist_genre', 
                   'popularity_cat', 'speechiness_cat', 
                   'danceability_cat', 'energy_cat', 
                   'instrumentalness_cat', 'liveness_cat', 
                   'valence_cat', 'tempo_cat', 
                   'duration_cat', 'decade')
song_mca_cat <- song[, qual_vars_cat]

# Réalisation de la MCA
mca_cat <- MCA(song_mca_cat, graph = FALSE)

# Visualisation des modalités sur le plan factoriel
fviz_mca_var(mca_cat, col.var = "cos2",
                   gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
                   repel = TRUE) +
  labs(title = "MCA - Variables catégorielles",
         subtitle = "Variables incluses: playlist_genre, popularity_cat, speechiness_cat, danceability_cat,\nenergy_cat, instrumentalness_cat, liveness_cat, valence_cat, tempo_cat,\nduration_cat, decade",
         caption = "Les couleurs indiquent la qualité de représentation (cos2) des modalités") +
  theme_minimal() +
  theme(legend.position = "bottom",
            plot.title = element_text(size = 16),
            plot.subtitle = element_text(size = 12, color = "gray40"),
            plot.caption = element_text(size = 10, color = "gray50"),
            axis.text = element_text(size = 12),
            axis.title = element_text(size = 14)) +
  guides(color = guide_colorbar(title = "Cos2", 
                                                title.position = "top",
                                                barwidth = 15,
                                                barheight = 1))

# Afficher avec une taille plus grande
options(repr.plot.width = 18, repr.plot.height = 16)

### Résultats

#### 1. **Axes factoriels**

##### **Dimension 1 (8.15%)** :

Cet axe semble opposer :

* **À droite** : des genres et époques associés à une musique plus **live**, **peu dansante**, **rock** et **années 1970–1980**, avec une tendance à la **longueur**.
* **À gauche** : des genres comme **EDM**, **rap**, ou **pop des années 2010–2020**, associés à des morceaux plus **courts**, **instrumentaux**, **très populaires**, avec **paroles dominantes**, souvent **joyeux**.

##### **Dimension 2 (7.14%)** :

Cet axe reflète plutôt une opposition d’ambiance :

* **En bas** : Musiques plus **live**, **peu dansantes**, parfois **instrumentales**, associées à des décennies anciennes (1950s–1980s).
* **En haut** : Musiques plus **rapides**, **énergétiques**, **avec paroles dominantes**, parfois **r\&b** ou **pop**.

> La projection est inversée selon la dim 1 par rapport aux résultats sous R. Cela n'affecte pas l'interprétation, seule l’orientation spatiale est différente.

---

#### 2. **Proximité des modalités**

* Les genres **rap**, **latin**, **court**, **très populaire** et **paroles dominantes** sont regroupés → musique courte, populaire, axée sur le texte.
* Le **rock** est fortement associé à la **décennie 1980**, à un **caractère live** et **peu dansant**.
* **EDM** est très proche de **instrumentale**, indiquant des morceaux sans paroles.
* Les catégories **"paroles dominantes"**, **"peu instrumental"**, et **"énergique"** sont toutes proches, suggérant que beaucoup de morceaux énergiques contiennent beaucoup de texte.
* **Pop des années 2010s et 2020s**, **dansants**, morceaux **moyennement long** se positionne plutôt au centre, ce qui reflète une certaine **polyvalence**.

---

#### 3. **Cos² (qualité de représentation)**

* Les modalités bien représentées par les deux axes incluent :

  * `rock`, `live`, `très populaire`, `joyeux`, `paroles dominantes`, `peu dansant`
* Les modalités avec des bulles plus petites comme `minor`, `pop`, `latin`, `r&b` sont moins bien représentées → elles nécessiteraient plus de dimensions pour être bien décrites.

---

### **Conclusion**

Cette MCA met en évidence des **clusters sémantiques clairs** entre les genres, l’époque, les caractéristiques émotionnelles et musicales :

* **Le rock des années 70–80** est clairement identifiable : live, peu dansant, majeur, long.
* **Le EDM et l’instrumental** vont de pair, souvent peu dansants, modernes, peu de texte.
* **Le rap** est court, populaire, avec des paroles dominantes, souvent dans un registre énergique mais pas exclusivement mineur ou majeur.
* **La pop des années 2010s–2020s** est très mixte : ni clairement joyeuse ni triste, ni très rapide ni lente → un profil "standard" qui touche un large public.

## Analyse Factorielle Multiple (MFA)

L’Analyse Factorielle Multiple (MFA) est une méthode d’analyse exploratoire multivariée qui permet d’étudier simultanément plusieurs groupes de variables de nature différente, notamment des variables quantitatives (caractéristiques audio) et qualitatives (genre, tonalité, mode, etc.). 

Dans le contexte de ce projet Spotify, la MFA est particulièrement pertinente car elle va permettre de :
- **Combiner** dans une même analyse les profils audio des morceaux et leurs attributs qualitatifs (genre, mode, key, etc.), en tenant compte de la structure de chaque groupe de variables.
- **Explorer les liens** entre les caractéristiques musicales objectives et les catégories musicales, pour voir par exemple si certains genres ou modes sont associés à des profils audio spécifiques.
- **Visualiser la structure globale** du jeu de données en intégrant toutes les dimensions importantes, ce qui offre une vision plus complète que l’ACP (centrée sur le quantitatif) ou la MCA (centrée sur le qualitatif) seules.

La MFA est donc un outil idéal pour comprendre comment les différentes facettes des chansons (audio et catégorielles) s’articulent et pour identifier des profils ou des clusters mixtes dans le dataset.

In [ ]:
# Analyse Factorielle Multiple (MFA) : audio + genre

# 1. Préparer les données pour MFA avec playlist_genre
mfa_data_genre <- song[, c("danceability", "energy", "loudness", "speechiness", "acousticness",
              "instrumentalness", "liveness", "valence", "tempo", "duration_s", "playlist_genre")]

# S'assurer que playlist_genre est un facteur
mfa_data_genre$playlist_genre <- as.factor(mfa_data_genre$playlist_genre)

# 2. Définir les groupes : 10 variables audio (quantitatives), 1 qualitative (playlist_genre)
group_genre <- c(10, 1)
type_genre <- c("s", "n")  # 1 groupe quanti, 1 groupe quali

# 3. Réaliser la MFA
mfa_genre <- MFA(mfa_data_genre, group = group_genre, type = type_genre, 
         name.group = c("Audio", "Genre"), graph = FALSE)

# 4. Visualisation des individus colorés par genre
fviz_mfa_ind(mfa_genre, habillage = mfa_data_genre$playlist_genre, palette = "Set2", 
       addEllipses = TRUE, ellipse.type = "confidence", label = "none") +
  labs(title = "MFA : projection des chansons selon audio et genre") +
  theme_minimal()


### Interprétation de l'Analyse Factorielle Multiple (MFA)

#### **Analyse audio + genre**

La MFA intégrant les caractéristiques audio et les genres musicaux révèle une **structuration claire** des genres selon leurs profils acoustiques :

**Séparation des genres :**
- Les genres forment des **clusters relativement distincts** dans l'espace factoriel, confirmant que les caractéristiques audio permettent une différenciation entre styles musicaux
- Certains genres comme **rock** et **edm** montrent une **séparation nette**, reflétant leurs profils acoustiques contrastés (rock : acoustique, live vs. edm : électronique, synthétique)
- D'autres genres comme **pop** et **latin** présentent une **plus grande diversité interne** et un chevauchement partiel, suggérant une variabilité stylistique au sein de ces catégories

**Axes factoriels :**
- Les axes révèlent des **oppositions claires** entre profils musicaux :
    - **Rock** : associé à des caractéristiques plus classiques (acousticness, liveness)
    - **EDM** : orienté vers des profils modernes et électroniques (energy, danceability)
    - **Rap** : caractérisé par des aspects rythmiques et vocaux (speechiness, tempo)


Ce graphique met en évidence la structuration des genres musicaux en fonction des caractéristiques audio. Il montre que certains genres (comme le rock et l'edm) sont bien séparés, tandis que d'autres (comme le pop et le latin) partagent des caractéristiques communes avec plusieurs genres.


In [ ]:
# Analyse Factorielle Multiple (MFA) : audio + decade

# 1. Préparer les données pour MFA avec decade
mfa_data_decade <- song[, c("danceability", "energy", "loudness", "speechiness", "acousticness",
                           "instrumentalness", "liveness", "valence", "tempo", "duration_s", "decade")]

# S'assurer que decade est un facteur
mfa_data_decade$decade <- as.factor(mfa_data_decade$decade)

# 2. Définir les groupes : 10 variables audio (quantitatives), 1 qualitative (decade)
group_decade <- c(10, 1)
type_decade <- c("s", "n")  # 1 groupe quanti, 1 groupe quali

# 3. Réaliser la MFA
mfa_decade <- MFA(mfa_data_decade, group = group_decade, type = type_decade, 
                  name.group = c("Audio", "Decade"), graph = FALSE)

# 4. Visualisation des individus colorés par décennie
fviz_mfa_ind(mfa_decade, habillage = mfa_data_decade$decade, palette = "Set1", 
             addEllipses = TRUE, ellipse.type = "confidence", label = "none") +
  labs(title = "MFA : projection des chansons selon audio et décennie") +
  theme_minimal()

#### **Analyse audio + décennie**

La MFA intégrant les caractéristiques audio et les décennies de sortie des morceaux révèle une **structuration temporelle claire** des chansons selon leurs profils acoustiques.

**Séparation des décennies :**
- Les décennies forment des **clusters relativement distincts** dans l'espace factoriel, confirmant que les caractéristiques audio évoluent au fil du temps
- Les morceaux des **années 1950s-1960s et 1970s** montrent une séparation nette
- Les morceaux des **années 2000s, 2010s et 2020s** présentent une plus grande diversité interne et un chevauchement partiel, suggérant une variabilité stylistique accrue dans les productions modernes

Ce graphique met en évidence l'**évolution des caractéristiques musicales au fil des décennies**. Il montre que les morceaux anciens (1950s-1970s) sont bien séparés des morceaux modernes (2000s-2020s), tandis que les décennies intermédiaires (1980s-1990s) servent de transition entre ces deux périodes.

In [ ]:
# Analyse Factorielle Multiple (MFA) : audio + popularity

# 1. Préparer les données pour MFA avec popularity_cat
mfa_data_popularity <- song[, c("danceability", "energy", "loudness", "speechiness", "acousticness",
                               "instrumentalness", "liveness", "valence", "tempo", "duration_s", "popularity_cat")]

# S'assurer que popularity_cat est un facteur
mfa_data_popularity$popularity_cat <- as.factor(mfa_data_popularity$popularity_cat)

# 2. Définir les groupes : 10 variables audio (quantitatives), 1 qualitative (popularity_cat)
group_popularity <- c(10, 1)
type_popularity <- c("s", "n")  # 1 groupe quanti, 1 groupe quali

# 3. Réaliser la MFA
mfa_popularity <- MFA(mfa_data_popularity, group = group_popularity, type = type_popularity, 
                     name.group = c("Audio", "Popularity"), graph = FALSE)

# 4. Visualisation des individus colorés par popularité
fviz_mfa_ind(mfa_popularity, habillage = mfa_data_popularity$popularity_cat, palette = "viridis", 
             addEllipses = TRUE, ellipse.type = "confidence", label = "none") +
  labs(title = "MFA : projection des chansons selon audio et popularité") +
  theme_minimal()

#### **Analyse audio + popularité**

La MFA intégrant les caractéristiques audio et les niveaux de popularité des morceaux révèle une structuration claire des chansons selon leur popularité.

**Séparation des niveaux de popularité :**

Les morceaux forment des clusters relativement distincts dans l'espace factoriel, confirmant que les caractéristiques audio influencent leur popularité.

Les morceaux **très populaires** montrent une concentration nette.

Les morceaux **peu populaires** présentent une plus grande diversité interne et un chevauchement partiel, suggérant une variabilité stylistique accrue ou des caractéristiques acoustiques et instrumentales.

Ce graphique met en évidence les relations entre les caractéristiques musicales et la popularité. Il montre que les morceaux très populaires se distinguent par des profils spécifiques, tandis que les morceaux peu populaires sont plus variés. Les morceaux de popularité moyenne occupent une position intermédiaire, suggérant que la popularité est influencée par une combinaison de caractéristiques audio et de facteurs contextuels.

## NMF

La Factorisation Matricielle Non-Négative (NMF) est une technique de réduction de dimension qui permet de décomposer une matrice en deux matrices de facteurs non négatifs. Dans le contexte de l'analyse des données musicales, NMF est particulièrement utile pour identifier des motifs latents ou des profils musicaux à partir des caractéristiques audio.

### Objectif de la NMF
L'objectif de la NMF dans ce projet est de découvrir des profils musicaux latents qui peuvent représenter des styles ou des genres musicaux spécifiques. En factorisant les données audio, nous espérons extraire des caractéristiques communes qui peuvent être utilisées pour la recommandation de musique ou pour mieux comprendre la structure des genres musicaux.

In [ ]:
library(NMF)
# 2. Sélection des colonnes audio uniquement depuis song
audio_features <- song %>%
  select(danceability, energy, loudness, speechiness, acousticness,
         instrumentalness, liveness, valence, tempo, duration_s) %>%
  na.omit()

# 3. Min-max scaling sur chaque colonne
audio_features <- as.data.frame(lapply(audio_features, function(x) (x - min(x)) / (max(x) - min(x))))

# 4. Conversion en matrice
audio_matrix <- as.matrix(audio_features)

# 5. Choix du nombre de composantes (ex. : 4 profils)
nmf_result <- nmf(audio_matrix, rank = 4, method = "brunet", nrun = 10, seed = 123)

# 6. Résumé
print(nmf_result)

# 7. Matrice W (coefficients pour chaque chanson)
W <- basis(nmf_result)

# 8. Matrice H (contributions de chaque feature à chaque profil)
H <- coef(nmf_result)

In [ ]:
# 1. Sélection des features audio
audio_features <- c('danceability', 'energy', 'loudness', 'speechiness',
                   'acousticness', 'instrumentalness', 'liveness',
                   'valence', 'tempo', 'duration_s')

X <- song[, audio_features]

# 2. Normalisation min-max (important pour la NMF)
X_scaled <- as.data.frame(lapply(X, function(x) (x - min(x)) / (max(x) - min(x))))
X_matrix <- as.matrix(X_scaled)

# 3. Tester plusieurs valeurs de r (nombre de composants)
errors <- c()
r_values <- seq(2, 10, by = 2)

for (r in r_values) {
  nmf_model <- nmf(X_matrix, rank = r, method = "brunet", nrun = 5, seed = 42)
  W <- basis(nmf_model)
  H <- coef(nmf_model)
  reconstruction <- W %*% H
  error <- norm(X_matrix - reconstruction, type = "F") # norme de Frobenius
  errors <- c(errors, error)
}

# 4. Tracer la courbe de l’erreur de reconstruction
plot(r_values, errors, type = "b", pch = 19,
     main = "Erreur de reconstruction vs nombre de composants (r)",
     xlab = "Nombre de composants (r)",
     ylab = "Erreur de reconstruction (norme de Frobenius)")
grid()

#On garde r=6 pour la suite soit 6 profils musicaux

# 5. Application de la NMF avec 6 composantes
nmf_model_6 <- nmf(X_matrix, rank = 6, method = "brunet", nrun = 10, seed = 42)
W_6 <- basis(nmf_model_6)
H_6 <- coef(nmf_model_6)

# 6. Créer un data.frame pour la matrice H
H_df <- as.data.frame(H_6)
colnames(H_df) <- audio_features
rownames(H_df) <- paste0("Profil ", 1:6)

# 7. Affichage de la matrice H
print("Matrice H (profils latents définis par les variables audio) :")

# 8. Visualisation : contribution des variables à chaque profil (heatmap)
library(reshape2)
library(ggplot2)
H_df_long <- melt(as.matrix(H_df))
colnames(H_df_long) <- c("Profil", "Variable", "Valeur")

ggplot(H_df_long, aes(x = Variable, y = Profil, fill = Valeur)) +
    geom_tile() +
    geom_text(aes(label = sprintf("%.2f", Valeur)), size = 3) +
    scale_fill_gradient(low = "#E7F6D5", high = "#2171B5") +
    labs(title = "Profils latents musicaux détectés par NMF (matrice H)",
             x = "Caractéristiques audio", y = "Profils NMF") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

On remarque que l'erreur de reconstruction décroit jusqu'à notre nombre variables totales, on choisit arbitrairement de prendre 6 facteurs pour la suite de l'analyse, ce qui va nous donner 6 profils musicaux.

### Résultats

#### 1. Profils latents détectés

L’analyse a révélé **6 profils musicaux latents** à partir des données. Chaque profil est défini par une combinaison unique de caractéristiques audio. Voici leur interprétation :

* **Profil 1 : Profil Instrumental**

  * Très forte **`instrumentalness`** avec très peu d'autres caractéristiques.
  * Correspond clairement à des morceaux **purement instrumentaux**, probablement **ambient**, **classique** ou **bande-son**.

* **Profil 2 : Profil Énergique-Intense**

  * Caractérisé par de **très fortes valeurs en `energy`, `loudness`, `tempo`, `duration` et `danceability`**.
  * Faible en composantes émotionnelles (`valence`), vocales et instrumentales.
  * Ce profil pourrait correspondre à des morceaux **très dynamiques et bruyants**, typiques de l’**EDM** (house, electro, techno) ou du **hard-rock**/**metal**.

* **Profil 3 : Profil Live**

  * Faibles valeurs générales sauf une **forte `liveness`**.
  * Peut représenter des morceaux **live**.

* **Profil 4 : Profil Émotionnel-Valence**

  * Faible en toutes dimensions sauf une très forte **`valence`**.
  * Ce profil regroupe des morceaux **émotionnellement très positifs**, avec une ambiance **joyeuse ou euphorique**, indépendamment du tempo ou de l’énergie.

* **Profil 5 : Profil Acoustique-Chant**

  * Dominé par une forte **`acousticness`**.
  * Ce profil, présente **`instrumentalness`=0**, et un faible **`speechiness`** mais différent de zéro.
  * Ce profil regroupe possiblement des morceaux **acoustiques et chantés**.

* **Profil 6 : Profil Rap-Rythmé**

  * Forte `danceability` et `speechiness`.
  * Ce profil semble capturer des morceaux **rythmés et très vocaux**, typiques du **rap**, **hip-hop**, voire certains morceaux **R\&B urbains**.

---

# Clustering


Dans cette partie, nous allons mettre en œuvre différentes méthodes de clustering afin d’identifier des groupes homogènes de morceaux au sein de notre jeu de données. L’objectif est de regrouper les morceaux présentant des caractéristiques similaires, sans utiliser d’information sur leur genre ou sous-genre, afin de révéler des structures ou des tendances cachées dans les données.  
Nous appliquerons plusieurs algorithmes de clustering non supervisé, tels que **K-Means**, le **mélange gaussien (GMM)** et la **classification ascendante hiérarchique (CAH)**. Nous comparerons ensuite les résultats obtenus et analyserons les profils des clusters identifiés, en les reliant aux variables musicales et aux genres présents dans le jeu de données.

>**IMPORTANT**  
> Dans ce notebook nous avons fait le choix de faire le clustering pur sur le jeu *data_songs* et ensuite nous allons projeter les clusters obtenus sur les genres des playlists. Ce choix a été fait afin de se concentrer principalement sur les caractéristiques audios des morceaux afin d'obtenir des profils musicaux. 

# Projection factorielle des individus (ACP)

On va d'abord afficher la représentation factorielle des individus pour pouvoir voir si on est capable de distinguer des classes naturelles. Pour cela, nous effectuons comme dans la partie précédente une ACP sur le jeu *data_songs*.

In [ ]:

# Sélection des variables numériques pour l'ACP sur onlysongs
onlysongs_pca_data <- onlysongs %>%
    select(track_popularity, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_s)

# Réalisation de l'ACP avec centrage et réduction
onlysongs_pca <- PCA(onlysongs_pca_data, scale.unit = TRUE, graph = TRUE)

# Affichage du pourcentage de variance expliquée par chaque composante principale
fviz_eig(onlysongs_pca, addlabels = TRUE, ylim = c(0, 40))

In [ ]:
# Affichage des individus sur les plans factoriels (Dim 1-2, 2-3, 1-3) pour l'ACP onlysongs
fviz_pca_ind(onlysongs_pca, 
             geom = "point", 
             col.ind = "cos2", # Coloration selon la qualité de représentation
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE) +
  labs(title = "Projection des individus sur le plan factoriel (Dim 1-2)") +
  theme_minimal()

fviz_pca_ind(onlysongs_pca, 
             axes = c(2, 3),
             geom = "point", 
             col.ind = "cos2",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE) +
  labs(title = "Projection des individus sur le plan factoriel (Dim 2-3)") +
  theme_minimal()

fviz_pca_ind(onlysongs_pca, 
             axes = c(1, 3),
             geom = "point", 
             col.ind = "cos2",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             repel = TRUE) +
  labs(title = "Projection des individus sur le plan factoriel (Dim 1-3)") +
  theme_minimal()
     

Nous obtenons la projection de tous les points sur les trois premières dimensions de la PCA calculée précédemment. Les points les plus au centre sont les moins bien représentés pour ces dimensions en général, tandis que ceux qui sont plus éloignés sont mieux représentés.

## K-Means
Avant d’appliquer l’algorithme de clustering K-Means, il est nécessaire de choisir le nombre optimal de clusters à utiliser.
Comme l’exploration visuelle précédente ne permet pas d’identifier clairement un nombre naturel de groupes, nous avons recours à la méthode du coude.  
Cette méthode consiste à faire varier le nombre de clusters et à observer l’évolution de l’inertie (distortion) pour déterminer le point à partir duquel ajouter des clusters n’apporte plus de gain significatif.
Le code suivant permet de visualiser ce critère et d’identifier le nombre de clusters optimal pour K-Means :

In [ ]:
# Vérifier le nombre de composantes principales disponibles
n_components <- ncol(onlysongs_pca$ind$coord)

# Méthode du coude (elbow method) pour déterminer le nombre optimal de clusters sur les coordonnées PCA de onlysongs
set.seed(42)
k_values <- 2:10
inertia_pca_onlysongs <- numeric(length(k_values))
# Utilisation des 7 premières composantes principales
pca_coords <- onlysongs_pca$ind$coord[, 1:n_components]

for (i in seq_along(k_values)) {
    k <- k_values[i]
    kmeans_result <- kmeans(pca_coords, centers = k, nstart = 25)
    inertia_pca_onlysongs[i] <- kmeans_result$tot.withinss
}

plot(k_values, inertia_pca_onlysongs, type = "b", pch = 19, col = "blue",
     xlab = "Nombre de clusters (k)", ylab = "Inertie intra-cluster",
     main = "Méthode du coude (K-means sur PCA onlysongs)",
     xlim = c(min(k_values) - 1, max(k_values) + 1))  # Élargir légèrement la plage de l'axe des abscisses

Le nombre de classes optimal pour cette méthode de clustering semble être **5 classes**. Nous allons donc conserver ce nombre pour la suite de notre analyse. Même si le coude ne semble pas être très visible, nous choisissons ce nombre de cluster en se basant sur le nombre choisi dans le notebook python. 

In [ ]:
# K-means sur les coordonnées PCA de onlysongs avec k = 5 clusters
set.seed(42)
kmeans_pca_onlysongs <- kmeans(onlysongs_pca$ind$coord[, 1:n_components], centers = 5, nstart = 25)

# Ajout des labels de cluster au dataset onlysongs
onlysongs$kmeans_pca_cluster <- as.factor(kmeans_pca_onlysongs$cluster)

# Visualisation des clusters sur les deux premières composantes principales
fviz_pca_ind(onlysongs_pca, 
             geom = "point", 
             col.ind = onlysongs$kmeans_pca_cluster, 
             palette = "jco", 
             addEllipses = TRUE) +
  labs(title = "K-means (k=5) sur ACP onlysongs (projection Dim 1-2)") +
  theme_minimal()

#affichage sur les dimensions principales 2-3
fviz_pca_ind(onlysongs_pca, 
             axes = c(2, 3),
             geom = "point", 
             col.ind = onlysongs$kmeans_pca_cluster, 
             palette = "jco", 
             addEllipses = TRUE) +
    labs(title = "K-means (k=5) sur ACP onlysongs (projection Dim 2-3)") +
    theme_minimal()

#affichage sur les dimensions principales 1-3
fviz_pca_ind(onlysongs_pca, 
             axes = c(1, 3),
             geom = "point", 
             col.ind = onlysongs$kmeans_pca_cluster, 
             palette = "jco", 
             addEllipses = TRUE) +
    labs(title = "K-means (k=5) sur ACP onlysongs (projection Dim 1-3)") +
    theme_minimal()

Ici, les clusters ne semblent pas bien se distinguer les uns des autres. Nous avons du mélange entre les clusters en fonction des dimensions.

Certaines classes semblent avoir une taille différente, pouvant indiquer un éventuel déséquilibre.   
Afin d'avoir une analyse plus complète de ces clusters, nous allons étudier les variables explicatives du jeu de données avec les clusters obtenus. 

**Étude des autres variables quantitatives avec nos clusters**  
On a plusieurs manières d'étudier les variables quantitatives dans nos clusters, soit avec des boxplots (assez visuels), soit avec une `heatmap`, cela permet d'avoir les moyennes centralisées et on a un visuel global de répartitions dans les clusters.

In [ ]:
# Passage au format long pour ggplot, en utilisant les clusters obtenus précédemment (kmeans_pca_cluster)
onlysongs_long <- melt(
    cbind(onlysongs[, c("kmeans_pca_cluster")], onlysongs_pca_data),
    id.vars = "kmeans_pca_cluster"
)

# Affichage des boxplots pour chaque variable par cluster
ggplot(onlysongs_long, aes(x = kmeans_pca_cluster, y = value, fill = kmeans_pca_cluster)) +
    geom_boxplot(outlier.size = 0.5) +
    facet_wrap(~ variable, scales = "free_y") +
    labs(title = "Boxplots des variables quantitatives par cluster (K-means PCA onlysongs)",
         x = "Cluster", y = "Valeur") +
    theme_minimal() +
    theme(legend.position = "none")

#agrandir beaucoup la taille des graphes
options(repr.plot.width=40, repr.plot.height=20)

Nous remarquons que certains clusters se distinguent plus pour certains caractéristiques de leurs musiques.
- Pour le **cluster 1**, il semble contenir seul un certain type de tempo, puisque la marge des tempos est très peu écartée contrairement aux autres clusters. Aussi, il contient beaucoup de morceaux avec une instrumentalness élevée.
- Pour le **cluster 2**, il contient les musiques avec un energy plus faible, ainsi qu'une acousticness élevée (qui sont corrélées négativement comme on l'a vu dans la partie exploratoire)
- Pour le **cluster 3**, il contient des musiques ayant une danceability plus faible.
- Pour le **cluster 4**, il se distingue par des sons ayant une speechiness élevée, donc des morceaux très parlés avec peu d'intruments.
- Pour le **cluster 5**, celui-ci contient les morceaux avec une valence plus élevée et une liveness faiblesse.

In [ ]:
# Fusionner song avec les clusters de onlysongs (sur track_name et track_artist)
song_with_cluster <- song %>%
    left_join(onlysongs[, c("track_name", "track_artist", "kmeans_pca_cluster")], 
              by = c("track_name", "track_artist"))

# Retirer les musiques qui n'ont pas de cluster
song_with_cluster_noNA <- song_with_cluster %>%
    filter(!is.na(kmeans_pca_cluster))

# Afficher un aperçu du résultat
head(song_with_cluster_noNA[, c("track_name", "track_artist", "kmeans_pca_cluster")])

#affichage du nombre de données total
cat("Nombre total de données dans song_with_cluster_noNA :", nrow(song_with_cluster_noNA), "\n")


In [ ]:
# Création d'une table de contingence : nombre de musiques genre et par cluster
heatmap_data <- song_with_cluster_noNA %>%
    group_by(kmeans_pca_cluster, playlist_genre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix <- reshape2::acast(heatmap_data, playlist_genre ~ kmeans_pca_cluster, value.var = "count", fill = 0)

# Création de la heatmap avec ggplot2 (format long)
heatmap_long <- as.data.frame(as.table(heatmap_matrix))
colnames(heatmap_long) <- c("playlist_genre", "kmeans_pca_cluster", "count")

ggplot(heatmap_long, aes(x = kmeans_pca_cluster, y = playlist_genre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par sous-genre et par cluster",
         x = "Cluster",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

Nous avons maintenant ajouté les genres pour voir leur présence dans les clusters. Nous allons analysé les genres des musiques en se basant sur l'analyse des clusters précédentes.
- **Cluster 1 :** Majorité de musiques du style EDM, on peut donc penser que ce style de musique possède souvent le même tempo.
- **Cluster 2 :** Majorité de sons R&B,  qui semble donc avoir une energy plus faible et une acousticness élevée.
- **Cluster 3 :** Beaucoup de sons rock, pop et edm. Ils semblent être liés à une danceability plus faible, ce qui peut être cohérent, puisque certains sons de ces genres ne sont pas fait pour danser.
- **Cluster 4 :** Majorité de musiques rap. Cohérent avec la speechiness très élevée puisque c'est un style de musique très parlé.
- **Cluster 5 :** présence de beaucoup de style de musique différents. Cela explique pourquoi dans l'analyse précédente on ne pouvait pas vraiment ressortir de caractéristiques.

In [ ]:
# Création d'une table de contingence : nombre de musiques par sous-genre et par cluster
heatmap_data <- song_with_cluster_noNA %>%
    group_by(kmeans_pca_cluster, playlist_subgenre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix <- reshape2::acast(heatmap_data, playlist_subgenre ~ kmeans_pca_cluster, value.var = "count", fill = 0)

# Création de la heatmap avec ggplot2 (format long)
heatmap_long <- as.data.frame(as.table(heatmap_matrix))
colnames(heatmap_long) <- c("playlist_subgenre", "kmeans_pca_cluster", "count")

ggplot(heatmap_long, aes(x = kmeans_pca_cluster, y = playlist_subgenre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par sous-genre et par cluster",
         x = "Cluster",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))


Nous avons mis la distribution des sous-genres dans les clusters. Les observations faites précédemment peuvent aussi s'appliquer sur les sous-genres les plus présents.

## Modèle de Mélange Gaussien (GMM)

Le modèle de mélange gaussien (GMM) est une méthode de clustering probabiliste qui permet de modéliser les données comme un mélange de plusieurs distributions gaussiennes. Contrairement à K-Means, GMM peut capturer des formes de clusters plus complexes et des variances différentes entre les clusters.

Nous allons appliquer le GMM sur les mêmes données que pour K-Means, en utilisant les trois premières composantes principales de l'ACP. Nous allons également utiliser le critère de BIC pour déterminer le nombre optimal de clusters.

In [ ]:
# Critère BIC pour déterminer le nombre optimal de composantes pour une GMM sur onlysongs_pca_data
library(mclust)

# Appliquer la GMM sur les données PCA (centrées et réduites)
gmm_model <- Mclust(onlysongs_pca_data, G = 1:10)  # G = nombre de clusters testés

# Afficher le BIC pour chaque nombre de clusters
plot(gmm_model, what = "BIC")

# Nombre optimal de clusters selon le BIC
cat("Nombre optimal de clusters selon le BIC :", gmm_model$G, "\n")

Nous affichons pour le GMM le nombre optimal de clusters. Ici, le nombre de clusters optimal d'après le critère BIC est de 9. Ce choix optimise le critère statistique, mais implique des groupes plus petits et une interprétation potentiellement plus complexe.

Nous avons donc décidé de retenir ce nombre de clusters pour la suite de l’analyse, sans chercher à simplifier davantage le modèle. Les analyses et visualisations suivantes seront donc réalisées avec 9 clusters, conformément à l’optimum donné par le BIC.

In [ ]:
# Appliquer la GMM sur les données PCA (centrées et réduites) avec 9 clusters
#Applique le modèle VEI (variance égale, covariance identique) sur les données PCA
gmm_9 <- Mclust(onlysongs_pca_data, G = 9, modelNames = "VEI")  # G = nombre de clusters

# Ajouter les labels de cluster GMM au dataset onlysongs
onlysongs$gmm_cluster <- as.factor(gmm_9$classification)

# Afficher un résumé du modèle GMM
summary(gmm_9)

# Afficher la répartition des chansons par cluster GMM
table(onlysongs$gmm_cluster)

# Affichage des clusters GMM sur les dimensions principales 1-2 sans ellipses, juste des points colorés
fviz_pca_ind(onlysongs_pca, 
             geom = "point", 
             col.ind = onlysongs$gmm_cluster, 
             palette = "jco", 
             addEllipses = TRUE, # Pas d'ellipses
             pointshape = 19, 
             pointsize = 2, 
             alpha.ind = 0.7) +
  labs(title = "GMM (k=9) sur ACP onlysongs (projection Dim 1-2) - sans ellipses") +
  theme_minimal()

Comme précédemment, les clusters ne semblent pas très bien se séparer sur le premier plan de l'ACP. On va afficher les boxplots des caractéristiques pour voir si nous trouvons des profils type dans nos clusters.

In [ ]:
# Affichage des boxplots des variables quantitatives par cluster GMM

# Sélection des variables quantitatives à visualiser
quant_vars <- c("track_popularity", "danceability", "energy", "loudness", "speechiness", 
                "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_s")

# Fusionner les clusters GMM avec onlysongs pour avoir les variables quantitatives et le cluster
onlysongs_gmm_long <- melt(
    onlysongs[, c("gmm_cluster", quant_vars)],
    id.vars = "gmm_cluster"
)

# Affichage des boxplots pour chaque variable par cluster GMM
ggplot(onlysongs_gmm_long, aes(x = gmm_cluster, y = value, fill = gmm_cluster)) +
    geom_boxplot(outlier.size = 0.5) +
    facet_wrap(~ variable, scales = "free_y") +
    labs(title = "Boxplots des variables quantitatives par cluster GMM",
         x = "Cluster GMM", y = "Valeur") +
    theme_minimal() +
    theme(legend.position = "none")

Nous pouvons essayer de trouver des caractéristiques des différents clusters en utilisant ces boxplots.
- Cluster 1 : Basse energy mais haute acousticness (cohérent car corrélation négative)
- Cluster 2 : Haute instrumentalness, haute energy
- Cluster 3 : Haute valence, haute dancebility
- Cluster 4 : Haute speechiness
- Cluster 5 : Plus haut tempo que les autres clusters (sauf cluster 1), basse energy
- Cluster 6 : Intrumentalness moyenne, haute energy
- Cluster 7 : Basse danceability, haute loudness
- Cluster 8 : Haute instrumentalness
- Cluster 9 : Haute liveness

In [ ]:
# Fusionner les clusters GMM avec les données song (sur track_name et track_artist)
song_with_gmm_cluster <- song %>%
    left_join(onlysongs[, c("track_name", "track_artist", "gmm_cluster")], 
              by = c("track_name", "track_artist"))

# Afficher un aperçu du résultat
head(song_with_gmm_cluster[, c("track_name", "track_artist", "gmm_cluster")])

In [ ]:
#création d'une table de contingence : nombre de musiques par genre et par cluster
heatmap_data_gmm <- song_with_gmm_cluster %>%
    group_by(gmm_cluster, playlist_genre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix_gmm <- reshape2::acast(heatmap_data_gmm, playlist_genre ~ gmm_cluster, value.var = "count", fill = 0)

# Création de la heatmap avec ggplot2 (format long)
heatmap_long_gmm <- as.data.frame(as.table(heatmap_matrix_gmm))
colnames(heatmap_long_gmm) <- c("playlist_genre", "gmm_cluster", "count")
ggplot(heatmap_long_gmm, aes(x = gmm_cluster, y = playlist_genre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par genre et par cluster GMM",
         x = "Cluster",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

In [ ]:
# Création d'une table de contingence : nombre de musiques par sous-genre et par cluster GMM
heatmap_data_gmm <- song_with_gmm_cluster %>%
    filter(!is.na(gmm_cluster)) %>%
    group_by(gmm_cluster, playlist_subgenre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix_gmm <- reshape2::acast(heatmap_data_gmm, playlist_subgenre ~ gmm_cluster, value.var = "count", fill = 0)

# Format long pour ggplot2
heatmap_long_gmm <- as.data.frame(as.table(heatmap_matrix_gmm))
colnames(heatmap_long_gmm) <- c("playlist_subgenre", "gmm_cluster", "count")

ggplot(heatmap_long_gmm, aes(x = gmm_cluster, y = playlist_subgenre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par sous-genre et par cluster GMM",
         x = "Cluster GMM",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

Nous pouvons essayer avec ces tables de contingence de lier l'analyse des propriétés des clusters avec les sous-genres de musiques.
- Cluster 1 : Neo Soul, urban contemporary et indie poptism, donc haute acousticness cohérente
- Cluster 2 : Progressive Electro House, electro house, qui est un style avec beaucoup d'energy
- Cluster 3 : Latin hip hop, latin pop, reggaeton, sont bien des musiques très dansantes, très positives
- Cluster 4 : Gangster rap, southern hip hop, hip hop, style de musique très parlée
- Cluster 5 : New jack swing, neo soul, classic rock, urban contemporary, pas énormément de lien avec une basse energy mais pas incohérent
- Cluster 6 : Electro house, progressive electro house, bien caractérisé par des musiques énergiques
- Cluster 7 : Hard rock, progresive electro house, pop edm, qui sont bien des musiques bruyantes et peu dansantes
- Cluster 8 : Progressive Electro House, electro house, très similaire au cluster 2
- Cluster 9 : Album rock, qui peut être des musiques issues de concerts

# CAH

Nous avons ensuite décidé de faire une méthode de clustering CAH, en traçant un dendrogramme par lien de Ward sur un échantillon de 10000 musiques prises aléatoirement. Avec la totalité de l'effectif, le temps de calcul est très long, nous avons donc pris la décision de ne faire ce clustering que sur un tiers des données, en espérant avoir des résultats tout de même concluants.

In [ ]:
# Échantillonnage aléatoire de 10000 morceaux pour accélérer la CAH
set.seed(42)
sample_indices <- sample(nrow(onlysongs_pca_data), min(10000, nrow(onlysongs_pca_data)))
onlysongs_pca_sample <- onlysongs_pca_data[sample_indices, ]

# Calcul de la matrice de distance euclidienne
dist_mat <- dist(onlysongs_pca_sample, method = "euclidean")

# CAH avec la méthode de Ward
cah_ward <- hclust(dist_mat, method = "ward.D2")

# Affichage du dendrogramme
plot(cah_ward, labels = FALSE, main = "Dendrogramme CAH (Ward)", xlab = "", sub = "")

Nous ne pouvons pas vraiment interpréter à vue d'oeil ce dendrogramme. Pour le couper ici, on devrait choisir l'endroit où nous avons le plus grand saut, en l'occurence à 2 clusters. Malgré tout, nous avons décidé de choisir 3 clusters, ce qui reste un grand saut. Aussi, nous avons fait ce choix pour garder une cohérence avec le notebook Python. 

In [ ]:
# Appliquer la CAH sur l'échantillon et assigner les clusters uniquement à cet échantillon
n_clusters <- 3
ward_clusters_sample <- cutree(cah_ward, k = n_clusters)

# Créer un vecteur de clusters pour tous les individus (NA par défaut)
ward_cluster_full <- rep(NA, nrow(onlysongs))
ward_cluster_full[sample_indices] <- ward_clusters_sample
onlysongs$ward_cluster <- as.factor(ward_cluster_full)

# Visualisation des clusters Ward (sur l'échantillon) sur les deux premières composantes principales de l'ACP
fviz_pca_ind(onlysongs_pca, 
             geom = "point", 
             col.ind = onlysongs$ward_cluster, 
             palette = "jco", 
             addEllipses = TRUE,
             select.ind = list(name = sample_indices)) +
  labs(title = "Classification hiérarchique de Ward (k=3) sur ACP onlysongs (projection Dim 1-2)") +
  theme_minimal()

Nous ne pouvons pas non plus avoir une bonne interprétation des clusters dans ce plan, puisque les clusters semblent empilés les uns sur les autres. Par conséquent, comme précédemment, nous allons regarder les caractéritiques de chaque cluster.

In [ ]:
# Fusionner les clusters Ward avec les données song (sur track_name et track_artist)
song_with_ward_cluster <- song %>%
    left_join(onlysongs[, c("track_name", "track_artist", "ward_cluster")], 
              by = c("track_name", "track_artist"))
# Afficher un aperçu du résultat
head(song_with_ward_cluster[, c("track_name", "track_artist", "ward_cluster")])

In [ ]:
# Affichage des boxplots des variables quantitatives par cluster Ward (CAH)

# Sélection des variables quantitatives à visualiser
quant_vars <- c("track_popularity", "danceability", "energy", "loudness", "speechiness", 
                "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_s")

# Passage au format long pour ggplot, en utilisant les clusters Ward
onlysongs_ward_long <- melt(
    onlysongs[, c("ward_cluster", quant_vars)],
    id.vars = "ward_cluster"
)

# Affichage des boxplots pour chaque variable par cluster Ward, sans les cluster NA
onlysongs_ward_long <- onlysongs_ward_long %>%
    filter(!is.na(ward_cluster))
ggplot(onlysongs_ward_long, aes(x = ward_cluster, y = value, fill = ward_cluster)) +
    geom_boxplot(outlier.size = 0.5) +
    facet_wrap(~ variable, scales = "free_y") +
    labs(title = "Boxplots des variables quantitatives par cluster Ward (CAH)",
         x = "Cluster Ward", y = "Valeur") +
    theme_minimal() +
    theme(legend.position = "none")

Malheureusement, on ne voit pas de grandes différences entre les clusters avec cette méthode. Elle ne semble pas vraiment concluante par rapport aux deux autres. Malgré tout, la plus grande différence se remarque entre les tempos, ce qui n'est pas très intéressant à analyser pour caractériser des styles de musiques.

In [ ]:
#création d'une table de contingence : nombre de musiques par genre et par cluster
heatmap_data_ward <- song_with_ward_cluster %>%
    group_by(ward_cluster, playlist_genre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix_ward <- reshape2::acast(heatmap_data_ward, playlist_genre ~ ward_cluster, value.var = "count", fill = 0)

# Création de la heatmap avec ggplot2 (format long)
heatmap_long_ward <- as.data.frame(as.table(heatmap_matrix_ward))
colnames(heatmap_long_ward) <- c("playlist_genre", "ward_cluster", "count")

ggplot(heatmap_long_ward, aes(x = ward_cluster, y = playlist_genre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par genre et par cluster Ward",
         x = "Cluster",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

In [ ]:
# Création d'une table de contingence : nombre de musiques par sous-genre et par cluster Ward
heatmap_data_ward <- song_with_ward_cluster %>%
    group_by(ward_cluster, playlist_subgenre) %>%
    summarise(count = n(), .groups = "drop")

# Conversion en matrice pour la heatmap
heatmap_matrix_ward <- reshape2::acast(heatmap_data_ward, playlist_subgenre ~ ward_cluster, value.var = "count", fill = 0)

# Format long pour ggplot2
heatmap_long_ward <- as.data.frame(as.table(heatmap_matrix_ward))
colnames(heatmap_long_ward) <- c("playlist_subgenre", "ward_cluster", "count")

ggplot(heatmap_long_ward, aes(x = ward_cluster, y = playlist_subgenre, fill = count)) +
    geom_tile(color = "white") +
    geom_text(aes(label = count), color = "black", size = 10) +
    scale_fill_gradient(low = "white", high = "steelblue") +
    labs(title = "Nombre de musiques par sous-genre et par cluster Ward",
         x = "Cluster Ward",
         y = "Sous-genre",
         fill = "Nombre") +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1, size = 20),
          axis.text.y = element_text(size = 20))

Ce manque de distinction se retrouve à nouveau dans la répartition des styles de musiques dans les clusters, où on ne voit pas de style qui ressort plus.

## Comparaison des méthodes de clustering

Dans cette partie, nous appliquons une Analyse des Correspondances Multiples (MCA) pour comparer les résultats des différentes méthodes de clustering (K-Means, GMM, CAH). 
L’objectif est de visualiser les clusters obtenus par chaque méthode dans un espace factoriel commun, afin d’évaluer leur cohérence et leur séparation.

In [ ]:
# Réduction des modalités rares pour 'track_artist'
freq_artists <- table(song$track_artist)
top_artists <- names(freq_artists[freq_artists > 100])
song$track_artist_reduced <- ifelse(song$track_artist %in% top_artists, as.character(song$track_artist), "Other")

# Réduction des modalités rares pour 'track_album_name'
freq_albums <- table(song$track_album_name)
top_albums <- names(freq_albums[freq_albums > 100])
song$track_album_name_reduced <- ifelse(song$track_album_name %in% top_albums, as.character(song$track_album_name), "Other")

# Colonnes catégorielles à utiliser pour la MCA
cols_cat_reduced <- c("track_artist_reduced", "track_album_name_reduced", "playlist_name", 
                      "playlist_genre", "playlist_subgenre", "key", "mode")

# Prendre un échantillon pour limiter la mémoire
set.seed(42)
sample_indices <- sample(nrow(song), 5000)
data_sample <- song[sample_indices, ]

# Vérifier les valeurs manquantes
colSums(is.na(data_sample[, cols_cat_reduced]))

# Nettoyer et convertir en facteur
data_sample_clean <- data_sample[, cols_cat_reduced]
data_sample_clean[] <- lapply(data_sample_clean, as.factor)

# MCA avec FactoMineR
mca <- MCA(data_sample_clean, ncp = 2, graph = FALSE)

In [ ]:
library(cluster)
library(mclust)
library(factoextra)

# Récupérer les coordonnées MCA correctement
X <- mca$ind$coord[, 1:2]  # Utiliser les coordonnées des individus de la MCA

# KMeans avec 5 clusters
set.seed(42)
kmeans_res <- kmeans(X, centers = 5, nstart = 25)
labels_kmeans <- kmeans_res$cluster

# GMM avec 10 clusters
gmm_res <- Mclust(X, G = 10)
labels_gmm <- gmm_res$classification

# CAH avec 3 clusters
hc <- hclust(dist(X), method = "ward.D2")
labels_cah <- cutree(hc, k = 3)

# Calcul des scores silhouette
sil_kmeans <- silhouette(labels_kmeans, dist(X))
sil_gmm <- silhouette(labels_gmm, dist(X))
sil_cah <- silhouette(labels_cah, dist(X))

cat("KMeans (5) Silhouette:", mean(sil_kmeans[, 3]), "\n")
cat("GMM (10) Silhouette:", mean(sil_gmm[, 3]), "\n")
cat("CAH (3) Silhouette:", mean(sil_cah[, 3]), "\n")

Avec analyse des silhouette scores pour les différentes méthodes de clustering que nous avons utilisées, la méthode présentant le meilleur score est K-means.

In [ ]:
# Calcul des indices de validation pour chaque méthode
library(clusterCrit)

# Silhouette déjà calculé précédemment
silhouette_scores <- c(
    mean(silhouette(labels_kmeans, dist(X))[, 3]),
    mean(silhouette(labels_gmm, dist(X))[, 3]),
    mean(silhouette(labels_cah, dist(X))[, 3])
)

# Calinski-Harabasz et Davies-Bouldin
ch_scores <- c(
    intCriteria(X, as.integer(labels_kmeans), c("Calinski_Harabasz"))$calinski_harabasz,
    intCriteria(X, as.integer(labels_gmm), c("Calinski_Harabasz"))$calinski_harabasz,
    intCriteria(X, as.integer(labels_cah), c("Calinski_Harabasz"))$calinski_harabasz
)
db_scores <- c(
    intCriteria(X, as.integer(labels_kmeans), c("Davies_Bouldin"))$davies_bouldin,
    intCriteria(X, as.integer(labels_gmm), c("Davies_Bouldin"))$davies_bouldin,
    intCriteria(X, as.integer(labels_cah), c("Davies_Bouldin"))$davies_bouldin
)

# Data frame des scores
df_scores <- data.frame(
    Méthode = c("KMeans (5)", "GMM (10)", "CAH (3)"),
    Silhouette = round(silhouette_scores, 3),
    Calinski_Harabasz = round(ch_scores, 3),
    Davies_Bouldin = round(db_scores, 3)
)


# Préparer les données pour ggplot
plot_data <- data.frame(
    X1 = X[, 1], X2 = X[, 2],
    KMeans = as.factor(labels_kmeans),
    GMM = as.factor(labels_gmm),
    CAH = as.factor(labels_cah)
)

p1 <- ggplot(plot_data, aes(x = X1, y = X2, color = KMeans)) +
    geom_point(size = 1) +
    labs(title = "KMeans (5 clusters)", x = "MCA Dim 1", y = "MCA Dim 2") +
    theme_minimal() + theme(legend.position = "none")

p2 <- ggplot(plot_data, aes(x = X1, y = X2, color = GMM)) +
    geom_point(size = 1) +
    labs(title = "GMM (10 clusters)", x = "MCA Dim 1", y = "MCA Dim 2") +
    theme_minimal() + theme(legend.position = "none")

p3 <- ggplot(plot_data, aes(x = X1, y = X2, color = CAH)) +
    geom_point(size = 1) +
    labs(title = "CAH (3 clusters)", x = "MCA Dim 1", y = "MCA Dim 2") +
    theme_minimal() + theme(legend.position = "none")

options(repr.plot.width=40, repr.plot.height=20)
p1 <- p1 + coord_fixed()
p2 <- p2 + coord_fixed()
p3 <- p3 + coord_fixed()

grid.arrange(p1, p2, p3, ncol = 3)
print(df_scores)

En visualisant la projection de nos clusters dans les dimensions de la MCA, nous pouvons également voir que la méthode K-means semble être la meilleure méthode de clustering.
En effet, les points semblent se répartir en 5 groupes de points, et ces groupes sont mieux représentés sur le K-means.

Pour la méthode GMM, certains groupements ne semblent pas présenter le même nombre de points, ce qui est un point négatif dans la répartition en clusters.

Pour la méthode CAH, il y a trop peu de clusters pour que les groupes soient bien clusterisés.

In [ ]:
# Analyse des Correspondances (CA) entre clusters KMeans, sous-genres et genres

# Table de contingence : cluster KMeans x sous-genre x genre

# Table cluster x sous-genre
tab_cluster_subgenre <- table(song_with_cluster_noNA$kmeans_pca_cluster, song_with_cluster_noNA$playlist_subgenre)
# Table cluster x genre
tab_cluster_genre <- table(song_with_cluster_noNA$kmeans_pca_cluster, song_with_cluster_noNA$playlist_genre)

# CA sur cluster x sous-genre
ca_subgenre <- CA(tab_cluster_subgenre, graph = FALSE)
# CA sur cluster x genre
ca_genre <- CA(tab_cluster_genre, graph = FALSE)

# Préparation des données pour ggplot (CA cluster x sous-genre)
coord_cluster <- as.data.frame(ca_subgenre$row$coord)
coord_cluster$Type <- "Cluster"
coord_cluster$Label <- rownames(coord_cluster)

coord_subgenre <- as.data.frame(ca_subgenre$col$coord)
coord_subgenre$Type <- "Sous-genre"
coord_subgenre$Label <- rownames(coord_subgenre)

coord_genre <- as.data.frame(ca_genre$col$coord)
coord_genre$Type <- "Genre"
coord_genre$Label <- rownames(coord_genre)

plot_data <- rbind(
    coord_cluster[, c("Dim 1", "Dim 2", "Type", "Label")],
    coord_subgenre[, c("Dim 1", "Dim 2", "Type", "Label")],
    coord_genre[, c("Dim 1", "Dim 2", "Type", "Label")]
)

# Couleurs pour chaque type
type_colors <- c("Cluster" = "#E41A1C", "Sous-genre" = "#377EB8", "Genre" = "#4DAF4A")

# Affichage du graphique
ggplot(plot_data, aes(x = `Dim 1`, y = `Dim 2`, color = Type, shape = Type)) +
    geom_point(size = 8, alpha = 0.8) +
    geom_text(aes(label = Label), size = 7, vjust = -1, fontface = "bold") +
    scale_color_manual(values = type_colors) +
    scale_shape_manual(values = c(16, 17, 15)) +
    labs(title = "Analyse des Correspondances : Clusters KMeans, Sous-genres et Genres",
             x = "Dimension 1", y = "Dimension 2", color = "Type", shape = "Type") +
    theme_minimal(base_size = 22) +
    theme(legend.position = "top")


Nous avons ici décidé de nous concentrer sur la représentation CA de la méthode de clustering la plus performante, qui est la méthode K-means.

On retrouve dans cette représentation les résultats que nous avons obtenu lors de l'analyse des tableaux de contingence de cette méthode.

- Cluster 1 : Proche du style EDM, notamment des sous-genres progressive electro house et electro house. Aussi proche du sous-genre big room, que nous ne voyions pas dans le tableau.
- Cluster 2 : Proche du R&B, mais aussi un peu du style latin, avec les sous-genres latin pop et new jack swing, qui n'était pas les plus importantes dans la tableau. En revanche, nous retrouvons bien les styles neo soul et urban contemporary.
- Cluster 3 : A nouveau, plus proche des styles EDM, rock et pop. Très proche de l'hard rock.
- Cluster 4 : Proche du rap, notamment du southern hip hop et du gangster rap.
- Cluster 5 : Dans le tableau de contingence, ce cluster contenant beaucoup de style de musiques. Cela se retrouve également dans la CA, mais le cluster est plus proche du style latin. 

# LDA

Nous allons maintenant appliquer une analyse discriminante linéaire (LDA) pour tenter de prédire les genres musicaux à partir des caractéristiques audio des morceaux. LDA est une méthode de classification qui cherche à maximiser la séparation entre les classes en projetant les données dans un espace de dimension inférieure.

In [ ]:
# Sélection des variables quantitatives et de la variable cible (playlist_genre) pour la LDA
library(MASS)
library(dplyr)

lda_data <- song %>%
  dplyr::select(playlist_genre, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_s, track_popularity)

#Normalisation des variables quantitatives
lda_data[, -1] <- scale(lda_data[, -1])

# S'assurer que la variable cible est un facteur
lda_data$playlist_genre <- as.factor(lda_data$playlist_genre)

# Appliquer la LDA
lda_model <- lda(playlist_genre ~ ., data = lda_data)

# Prédire les coordonnées des individus sur les axes discriminants
lda_pred <- predict(lda_model)

# Afficher un résumé du modèle LDA
print(lda_model)

Nous remarquons dans un premier temps que les genres de musiques sont représentés à peu près dans la même proportion, mais le style EDM suivi du rap restent les styles de musiques les plus présents.

Après normalisation des données, nous pouvons voir que certaines caractéritiques semblent caractériser certains style de musiques :
- edm : Haute energy,  basse acousticness
- latin : Haute danceability, haute valence
- pop : Pas vraiment de caractéristiques qui se démarquent du reste
- r&b : Basse energy, haute acousticness
- rap : Haute danceability, haute speechiness
- rock : Basse danceability, basse speechiness

In [ ]:

lda_df <- data.frame(lda_pred$x, playlist_genre = lda_data$playlist_genre)

# Calcul des barycentres pour chaque genre
centroids <- lda_df %>%
       group_by(playlist_genre) %>%
       summarise(LD1 = mean(LD1), LD2 = mean(LD2))

# Calcul des coefficients des variables (loadings) pour LD1 et LD2
lda_loadings <- as.data.frame(lda_model$scaling[, 1:2])
lda_loadings$variable <- rownames(lda_loadings)
colnames(lda_loadings)[1:2] <- c("LD1", "LD2")

# Mise à l'échelle des flèches pour une meilleure visibilité
arrow_scale <- 2
lda_loadings$LD1 <- lda_loadings$LD1 * arrow_scale
lda_loadings$LD2 <- lda_loadings$LD2 * arrow_scale

# Affichage du scatterplot avec barycentres et flèches des variables
ggplot(lda_df, aes(x = LD1, y = LD2, color = playlist_genre)) +
       geom_point(alpha = 0.5) +
       geom_point(data = centroids, aes(x = LD1, y = LD2, fill = playlist_genre), 
                        size = 8, shape = 21, color = "black", show.legend = FALSE, inherit.aes = FALSE) +
       geom_text(data = centroids, aes(x = LD1, y = LD2, label = playlist_genre), 
                       color = "black", size = 5, vjust = -1, inherit.aes = FALSE) +
       geom_segment(data = lda_loadings, aes(x = 0, y = 0, xend = LD1, yend = LD2),
                             arrow = arrow(length = unit(0.3, "cm")), color = "black", inherit.aes = FALSE) +
       geom_text(data = lda_loadings, aes(x = LD1, y = LD2, label = variable),
                       color = "black", size = 5, vjust = 1.2, inherit.aes = FALSE) +
       labs(title = "Projection LDA : individus, barycentres et vecteurs des variables",
               x = "LD1", y = "LD2", color = "Genre") +
       theme_minimal()

Nous avons projeté les individus sur les premiers axes discriminants de LDA, en ajoutant les barycentres des classes de genre et les variables explicatives. Nous pouvons voir que certains genres se démarquent un peu des autres dans le nuage de points. Nous avons l'EDM qui se distingue plus vers le haut du graphe, le rap en bas à droite et le rock en bas à gauche. Nous pouvons aussi dire que le rap se distingue un peu en bas du nuage. Pour ce qui est de la pop et du latin, comme nous avons vu précédemment, ils ne se distinguent pas vraiment avec des caractéristiques très particulières. Par conséquent, ils sont très au centre et mélangés avec des individus d'autres genres.

Nous pouvons aussi lier à certaines caractéristiques nos styles de musique à nouveau. Nos analyses seront très similaires aux précédentes. Le rock se situe à l'opposé de la flèche de danceability et de speechiness, ce n'est donc pas un style ni dansant ni parlé, contrairement au rap. Nous retrouvons l'EDM qui est très instrumental et très énergétique. Le R&B semble être un style plutôt acoustique, et parlé. Finalement pour la pop et le latin, qui se trouvent au centre ddu graphe, ils ne semblent pas faire ressortir de caractéristiques particulières.

Nous avons ensuite essayé de faire une LDA pour voir si la popularité d'un titre est liée à ses caractéristiques intrinsèques.

In [ ]:

# LDA pour estimer la popularité à partir des variables quantitatives
library(MASS)
library(dplyr)

# Création d'une variable catégorielle de popularité (par exemple, 5 classes : 0;20 , 20;40, 40;60, 60;80, 80;100)
song$popularity_class <- cut(
    song$track_popularity,
    breaks = quantile(song$track_popularity, probs = seq(0, 1, by = 0.2)),
    labels = c("Très faible", "Faible", "Moyenne", "Forte", "Très forte"),
    include.lowest = TRUE
)

# Sélection des variables quantitatives et de la variable cible
lda_pop_data <- song %>%
    dplyr::select(popularity_class, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_s)

# Normalisation des variables quantitatives
lda_pop_data[, -1] <- scale(lda_pop_data[, -1])

# S'assurer que la variable cible est un facteur
lda_pop_data$popularity_class <- as.factor(lda_pop_data$popularity_class)

# Appliquer la LDA
lda_pop_model <- lda(popularity_class ~ ., data = lda_pop_data)

# Afficher un résumé du modèle LDA
print(lda_pop_model)

A première vue, il n'y pas de caractéristiques quantitatives qui permettent à des musiques plus ou moins populaires de se distinguer des autres. Nous allons malgré tout regarder si la projection des individus sur le premier plan de la LDA permet d'avoir des résultats plus concluants.

In [ ]:
# Scatterplot des axes LDA (premiers axes discriminants)
lda_pop_pred <- predict(lda_pop_model)
lda_pop_df <- as.data.frame(lda_pop_pred$x)
lda_pop_df$popularity_class <- lda_pop_data$popularity_class

In [ ]:
# Calcul des barycentres pour chaque classe de popularité
centroids_pop <- lda_pop_df %>%
    group_by(popularity_class) %>%
    summarise(LD1 = mean(LD1), LD2 = mean(LD2))

# Calcul des coefficients des variables (loadings) pour LD1 et LD2
lda_pop_loadings <- as.data.frame(lda_pop_model$scaling[, 1:2])
lda_pop_loadings$variable <- rownames(lda_pop_loadings)
colnames(lda_pop_loadings)[1:2] <- c("LD1", "LD2")

# Mise à l'échelle des flèches pour une meilleure visibilité
arrow_scale <- 2
lda_pop_loadings$LD1 <- lda_pop_loadings$LD1 * arrow_scale
lda_pop_loadings$LD2 <- lda_pop_loadings$LD2 * arrow_scale

# Affichage du scatterplot avec barycentres et flèches des variables
ggplot(lda_pop_df, aes(x = LD1, y = LD2, color = popularity_class)) +
    geom_point(alpha = 0.5) +
    geom_point(data = centroids_pop, aes(x = LD1, y = LD2, fill = popularity_class), 
               size = 8, shape = 21, color = "black", show.legend = FALSE, inherit.aes = FALSE) +
    geom_text(data = centroids_pop, aes(x = LD1, y = LD2, label = popularity_class), 
              color = "black", size = 5, vjust = -1, inherit.aes = FALSE) +
    geom_segment(data = lda_pop_loadings, aes(x = 0, y = 0, xend = LD1, yend = LD2),
                 arrow = arrow(length = unit(0.3, "cm")), color = "black", inherit.aes = FALSE) +
    geom_text(data = lda_pop_loadings, aes(x = LD1, y = LD2, label = variable),
              color = "black", size = 5, vjust = 1.2, inherit.aes = FALSE) +
    labs(title = "Projection LDA (popularité) : individus, barycentres et vecteurs des variables",
         x = "LD1", y = "LD2", color = "Popularité") +
    theme_minimal()

Malheureusement, après projection sur les dimensions de la LDA, et en plaçant les barycentres des classes de popularité, les individus restent très proches les uns des autres. A première vue, on pourrait penser que les musiques plus populaires sont bruyantes et peu énergétiques, mais le graphe ne permet pas une distinction assez précises pour arriver à cette conclusion.

## Système de recommandation

### Objectif
L'objectif de cette section est de développer un système de recommandation basé sur les profils musicaux latents identifiés par la NMF. Ce système permettra de :

1. Recommander des **morceaux à ajouter à une playlist existante** en analysant son profil musical global
2. Suggérer des **chansons similaires à un morceau spécifique choisi** par l'utilisateur

### Méthodologie
En se basant sur les 6 profils latents extraits (instrumental, énergique-intense, live, émotionnel-valence, acoustique-chant, rap-rythmé), le système calcule des mesures de similarité pour proposer des recommandations personnalisées et pertinentes.

1. **Extraction des profils latents** : Utilisation des 6 profils musicaux latents identifiés par la NMF.
2. **Calcul de similarité** : Mesures de similarité entre les morceaux ou le profil moyen d'une playlist et les profils latents pour déterminer les recommandations.
3. **Recommandation personnalisée** : Proposer des morceaux en fonction des similarités calculées.

Nous commençons par effectuer une recommandation à partir d'une playlist spécfique. Pour cela, nous allons utiliser une fonction qui prend en entrée une playlist et renvoie les morceaux similaires en fonction de leur profil musical. 

In [ ]:
# Système de recommandation basé sur les profils NMF

# 1. Ajouter les profils NMF comme colonnes au dataframe original
song_with_profiles <- song
song_with_profiles[, paste0("profile_", 1:6)] <- W_6

# 2. Fonction pour recommander des chansons basées sur une playlist
recommend_songs <- function(playlist_name, num_recommendations = 3) {
    # Vérifier si la playlist existe
    if(!(playlist_name %in% song_with_profiles$playlist_name)) {
        stop("Playlist non trouvée dans le dataset")
    }
    
    # Récupérer les morceaux de la playlist
    playlist_songs <- song_with_profiles[song_with_profiles$playlist_name == playlist_name, ]
    
    # Calculer le profil moyen de la playlist
    playlist_profile <- colMeans(playlist_songs[, paste0("profile_", 1:6)])
    
    # Exclure les morceaux déjà dans la playlist
    other_songs <- song_with_profiles[song_with_profiles$playlist_name != playlist_name, ]
    
    # Calculer la similarité (distance euclidienne) entre le profil de la playlist et chaque morceau
    similarities <- apply(other_songs[, paste0("profile_", 1:6)], 1, function(song_profile) {
        -sqrt(sum((song_profile - playlist_profile)^2))  # Négative car nous voulons maximiser
    })
    
    # Trier les morceaux par similarité décroissante
    other_songs$similarity <- similarities
    recommendations <- other_songs[order(other_songs$similarity, decreasing = TRUE), ]
    
    # Retourner les top N recommandations
    return(head(recommendations[, c("track_name", "track_artist", "playlist_genre", "playlist_subgenre", "similarity")], num_recommendations))
}


Maintenant nous allons faire une recommandation de morceaux à ajouter à une playlist existante avec la fonction `recommend_song` qui prend en entrée une playlist et renvoie les morceaux similaires en fonction du profil musical moyen de la playlist.

A partir de la playlist EDM House & Dance, nous allons extraire le profil musical moyen de cette playlist et recommander des morceaux similaires en fonction des profils musicaux latents. 

In [ ]:
# 3. Exemple d'utilisation
# Obtenir les noms de playlists uniques
unique_playlists <- unique(song_with_profiles$playlist_name)

# Sélectionner une playlist aléatoire pour démonstration
set.seed(123)
example_playlist <- sample(unique_playlists, 1)

# Afficher le nom de la playlist sélectionnée
cat("Playlist sélectionnée:", as.character(example_playlist), "\n\n")

# Afficher quelques chansons de la playlist sélectionnée
playlist_sample <- song_with_profiles[song_with_profiles$playlist_name == example_playlist, 
                                                                     c("track_name", "track_artist", "playlist_genre")]
cat("Exemple de chansons dans cette playlist:\n")
print(head(playlist_sample, 3))
cat("\n")

# Obtenir les recommandations sous la forme d'un tableau sans les playlist_subgenre
recommendations <- recommend_songs(example_playlist, num_recommendations = 3)
cat("Recommandations de chansons basées sur la playlist sélectionnée:\n")
print(recommendations[, -4])  # Exclure la colonne playlist_subgenre

L'algorithme nous propose les morceaux suivants :
* Think Before You Talk - Chad Cooper
* Make It To Heaven (with Raye) - Rework David Guetta
* 2 Hearts (feat. Gia Koka)    Sam Feldt

Après écoute, ces morceaux correspondent bien à l'esprit de la playlist EDM House & Dance. Ils sont électroniques, dansants et ont une ambiance similaire.

In [ ]:
# 4. Fonction pour recommander à partir d'une chanson spécifique
recommend_from_song <- function(track_name, track_artist, num_recommendations = 3) {
    # Trouver la chanson dans le dataset
    song_idx <- which(song_with_profiles$track_name == track_name & 
                                        song_with_profiles$track_artist == track_artist)
    
    if(length(song_idx) == 0) {
        stop("Chanson non trouvée dans le dataset")
    }
    
    # Récupérer le profil de la chanson
    song_profile <- as.numeric(song_with_profiles[song_idx[1], paste0("profile_", 1:6)])
    
    # Exclure la chanson elle-même
    other_songs <- song_with_profiles[-song_idx, ]
    
    # Calculer la similarité
    similarities <- apply(other_songs[, paste0("profile_", 1:6)], 1, function(profile) {
        -sqrt(sum((profile - song_profile)^2))
    })
    
    # Trier par similarité
    other_songs$similarity <- similarities
    recommendations <- other_songs[order(other_songs$similarity, decreasing = TRUE), ]
    
    # Retourner les top N recommandations
    return(head(recommendations[, c("track_name", "track_artist", "playlist_genre", "playlist_subgenre", "similarity")], num_recommendations))
}

In [ ]:
#Recommandation pour la chanson "Thunderstruck" de "AC/DC"
track_name <- "Thunderstruck"
track_artist <- "AC/DC"
num_recommendations <- 5
recommendations <- recommend_from_song(track_name, track_artist, num_recommendations)
cat("Recommandations pour la chanson 'Thunderstruck' de 'AC/DC':\n")
print(recommendations[, -4])  # Exclure la colonne playlist_subgenre

L'algorithme nous propose les morceaux suivants :
* Ita - Dyne Side
* Summer - Marshmello
* Bella Ciao - Hardwell

L'algorithme nous renvoie les morceaux les plus similaires au morceau choisi, en se basant sur les profils musicaux latents. Ce sont des morceaux qui partagent des caractéristiques audio avec `Thunderstruck`.

# Conclusion

Ce projet d'analyse de données musicales a permis d'explorer en profondeur un dataset riche de plus de 30 000 morceaux issus de playlists Spotify, en appliquant diverses techniques d'analyse multivariée et d'apprentissage non supervisé.

### Apports méthodologiques

**Maîtrise d'un pipeline complet d'analyse**  
Ce projet nous a permis de développer une expertise transversale en :
- **Préparation et nettoyage** de données complexes
- **Analyse exploratoire** approfondie avec visualisations avancées
- **Application coordonnée** de multiples techniques d'analyse multivariée
- **Validation et comparaison** rigoureuse des méthodes de clustering

**Expertise en techniques d'analyse multivariée**  
Nous avons acquis une maîtrise pratique de :
- L'**Analyse en Composantes Principales (ACP)** pour la réduction de dimensionnalité
- L'**Analyse des Correspondances Multiples (ACM)** pour les variables qualitatives
- La **Factorisation Matricielle Non-Négative (NMF)** pour l'extraction de profils latents
- Les **méthodes de clustering** (K-Means, GMM, CAH) avec optimisation des hyperparamètres

### Découvertes scientifiques

**Structures cachées dans les données musicales**  
L'analyse a révélé des **patterns significatifs** :
- **6 profils musicaux latents** distincts identifiés par NMF
- **Correspondances fortes** entre caractéristiques audio et genres musicaux
- **Segmentation naturelle** des morceaux en groupes homogènes

**Validation de la cohérence des résultats**  
La convergence des différentes méthodes confirme la **robustesse** de nos analyses :
- Les clusters correspondent aux genres attendus
- Les profils NMF sont musicalement interprétables
- La MCA révèle des associations logiques entre variables

### Compétences techniques développées

**Programmation avancée en Python**  
- Manipulation de **pandas** pour les données structurées
- Visualisation sophistiquée avec **matplotlib/seaborn**
- Implémentation d'algorithmes **scikit-learn** 

### Applications concrètes

Ce projet démontre notre capacité à :
- **Transformer** des données brutes en insights exploitables
- **Développer** des systèmes de recommandation fonctionnels

### Perspectives professionnelles

Cette expérience nous prépare à :
- **Analyser** tout type de données multidimensionnelles
- **Conseiller** sur le choix de méthodes analytiques appropriées
- **Communiquer** des résultats techniques

Ce projet illustre parfaitement comment l'analyse de données peut révéler des structures cachées dans des domaines complexes, tout en développant une expertise technique solide et une approche méthodologique rigoureuse.